# Limpieza del dataframe
Este es el archivo dónde se limpiará el dataframe, se harán transformaciones de los datos para limpiarlos pero no para explorarlos, eso aún no.

## Importación de librerías

In [1]:
from unidecode import unidecode
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import os
from ydata_profiling import ProfileReport
from rapidfuzz import process, fuzz

/home/diggspapu/.pyenv/versions/3.10.16/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Lectura del dataset relleno

In [ ]:
df = pd.read_csv(f'./dataset/violenciaIntrafamiliarGuatemala2008_2023.csv')

/tmp/ipykernel_212406/2472453134.py:1: DtypeWarning: Columns (45,46,48,51,52,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{os.getcwd()}/dataset/violenciaIntrafamiliarGuatemala2008_2023.csv')


## Explorar los datos preliminarmente

In [3]:
df.describe()

,AGR_EDAD,ANO_EMISION,DIA_EMISION,HEC_ANO,HEC_DIA,NUM_HIJ_HOM,NUM_HIJ_MUJ,OTRAS_VICTIMAS,TOTAL_HIJOS,VIC_EDAD,...,CANTIDAD_DIVISION_INVESTIGACION_CRIMINAL,CANTIDAD_CEMENTERIO,CANTIDAD_CENTRO_COMUNITARIO,CANTIDAD_CENTRO_ATENCION_PERMANENTE,CANTIDAD_CENTRO_SALUD,CANTIDAD_DIRECCION,CANTIDAD_ESCUELA,CANTIDAD_HOSPITAL,CANTIDAD_OTRO,CANTIDAD_PUESTO_SALUD
count,521955.000000,521955.000000,521955.000000,521955.000000,521955.000000,519573.000000,519573.000000,521955.000000,519573.000000,521955.000000,...,521955.000000,521955.000000,521955.000000,521955.000000,521955.000000,521955.000000,521955.000000,521955.000000,521955.000000,521955.0000
mean,697.465207,2015.698397,15.363625,2107.084814,16.098273,1879.473079,1879.108691,3828.291956,1880.323279,173.405284,...,39.847964,0.167187,0.427489,5.395005,7.463042,0.167187,0.355525,0.842979,3.155626,18.2004
std,2483.438146,4.600242,8.743369,849.409444,12.491222,3905.303818,3905.084954,4859.740255,3904.895085,1173.766592,...,61.635557,0.373143,1.450163,5.782975,7.038975,0.373143,0.478673,0.531015,3.273154,23.6608
min,7.000000,2008.000000,1.000000,200.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
25%,27.000000,2012.000000,8.000000,2012.000000,8.000000,0.000000,0.000000,0.000000,1.000000,24.000000,...,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.0000
50%,34.000000,2016.000000,15.000000,2016.000000,15.000000,1.000000,1.000000,2.000000,2.000000,31.000000,...,16.000000,0.000000,0.000000,3.000000,6.000000,0.000000,0.000000,1.000000,3.000000,14.0000
75%,43.000000,2020.000000,23.000000,2020.000000,23.000000,3.000000,3.000000,9999.000000,5.000000,40.000000,...,25.000000,0.000000,0.000000,8.000000,14.000000,0.000000,1.000000,1.000000,4.000000,21.0000
max,9999.000000,2023.000000,31.000000,9999.000000,99.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,...,176.000000,1.000000,7.000000,19.000000,21.000000,1.000000,1.000000,2.000000,13.000000,115.0000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521955 entries, 0 to 521954
Data columns (total 96 columns):
 #   Column                                                                     Non-Null Count   Dtype  
---  ------                                                                     --------------   -----  
 0   AGR_ALFAB                                                                  521955 non-null  object 
 1   AGR_DEDICA                                                                 106507 non-null  object 
 2   AGR_EDAD                                                                   521955 non-null  float64
 3   AGR_ESCOLARIDAD                                                            521955 non-null  object 
 4   AGR_EST_CIV                                                                521918 non-null  object 
 5   AGR_GRUPET                                                                 521955 non-null  object 
 6   AGR_NACIONAL                                

# Rellenar nulos

In [5]:
mcipios = []
mcipios.extend(list(df[df["DEPTO"].isnull() | df["HEC_DEPTO"].isnull()][["HEC_DEPTOMCPIO"]]\
            .rename({"MUNICIPIO":"HEC_DEPTOMCPIO"})
            .drop_duplicates()["HEC_DEPTOMCPIO"]))
mcipios.extend(list(df[df["DEPTO"].isnull() | df["HEC_DEPTO"].isnull()][["DEPTO_MCPIO"]]\
            .rename({"MUNICIPIO":"DEPTO_MCPIO"})\
            .drop_duplicates()["DEPTO_MCPIO"]))

In [6]:
mcipios = df[df["DEPTO"].isnull() | df["HEC_DEPTO"].isnull()][["HEC_DEPTO", "DEPTO", "DEPTO_MCPIO", "HEC_DEPTOMCPIO"]].drop_duplicates()
mcipios

,HEC_DEPTO,DEPTO,DEPTO_MCPIO,HEC_DEPTOMCPIO
17,NaN,NaN,san jose el golfo,san jose el golfo
19,NaN,NaN,san raimundo,san raimundo
84,NaN,NaN,petapa,petapa
90,guatemala,NaN,petapa,guatemala
103,NaN,guatemala,guatemala,petapa
...,...,...,...,...
479105,NaN,guatemala,guatemala,el chol
480314,totonicapan,NaN,santa lucia la reforma,momostenango
480974,NaN,huehuetenango,petatan,union cantinil
481350,peten,NaN,lanquin,sayaxche


In [7]:
municipios_departamentos = [
    ("San José el Golfo", "Guatemala"),
    ("San Raimundo", "Guatemala"),
    ("Petapa", "Guatemala"),
    ("Guatemala", "Guatemala"),
    ("Patulul", "Suchitepéquez"),
    ("Alotenango", "Sacatepéquez"),
    ("Comalapa", "Chimaltenango"),
    ("Yepocapa", "Chimaltenango"),
    ("San Rafael las Flores", "Santa Rosa"),
    ("San Lucas Toliman", "Sololá"),
    ("San Cristobal Totonicapán", "Totonicapán"),
    ("Santa Lucía La Reforma", "Totonicapán"),
    ("Ostuncalco", "Quetzaltenango"),
    ("Olintepeque", "Quetzaltenango"),
    ("Colomba", "Quetzaltenango"),
    ("Palestina de los Altos", "Quetzaltenango"),
    ("San José El Idolo", "Suchitepéquez"),
    ("San Pablo Jocopilas", "Suchitepéquez"),
    ("Sololá", "Sololá"),
    ("Samayac", "Suchitepéquez"),
    ("Rio Bravo", "Escuintla"),
    ("Mixco", "Guatemala"),
    ("San Rafael Pié de La Cuesta", "San Marcos"),
    ("El Rodeo", "San Marcos"),
    ("Ocós", "San Marcos"),
    ("Ixtahuacán", "Huehuetenango"),
    ("Unión Cantinil", "Huehuetenango"),
    ("Chichicastenango", "Quiché"),
    ("Uspantán", "Quiché"),
    ("Ixcán", "Quiché"),
    ("Momostenango", "Totonicapán"),
    ("Nebaj", "Quiché"),
    ("Tucurú", "Alta Verapaz"),
    ("Lanquín", "Alta Verapaz"),
    ("Cahabón", "Alta Verapaz"),
    ("Santa Catalina la Tinta", "Alta Verapaz"),
    ("Jocotán", "Chiquimula"),
    ("Quetzaltepeque", "Chiquimula"),
    ("Villa Nueva", "Guatemala"),
    ("Pochuta", "Chimaltenango"),
    ("Cuilapa", "Santa Rosa"),
    ("San Bartolo", "Totonicapán"),
    ("Santo Tomás La Unión", "Suchitepéquez"),
    ("Rio Blanco", "San Marcos"),
    ("Barillas", "Huehuetenango"),
    ("Chinique", "Quiché"),
    ("El Chol", "Baja Verapaz"),
    ("San Juan Erminta", "Chiquimula"),
    ("San Jacinto", "Chiquimula"),
    ("San José Pinula", "Guatemala"),
    ("Amatitlán", "Guatemala"),
    ("Patzicía", "Chimaltenango"),
    ("San Juan Cotzal", "Quiché"),
    ("Sacapulas", "Quiché"),
    ("Chajul", "Quiché"),
    ("Melchor de Mencos", "Petén"),
    ("La Unión", "Zacapa"),
    ("Soloma", "Huehuetenango"),
    ("Cunén", "Quiché"),
    ("San José Acatempa", "Jutiapa"),
    ("Antigua Guatemala", "Sacatepéquez"),
    ("Chimaltenango", "Chimaltenango"),
    ("Cabricán", "Quetzaltenango"),
    ("San Mateo Ixtatán", "Huehuetenango"),
    ("Río Hondo", "Zacapa"),
    ("Jerez", "Jutiapa"),
    ("San Miguel Dueñas", "Sacatepéquez"),
    ("Santa Cruz Balanyá", "Chimaltenango"),
    ("Escuintla", "Escuintla"),
    ("Quetzaltenango", "Quetzaltenango"),
    ("Nueva Concepción", "Escuintla"),
    ("Nentón", "Huehuetenango"),
    ("Panzós", "Alta Verapaz"),
    ("Santa Cruz del Quiché", "Quiché"),
    ("San Francisco El Alto", "Totonicapán"),
    ("San Juan Sacatepéquez", "Guatemala"),
    ("Chiquimula", "Chiquimula"),
    ("Nuevo Progreso", "San Marcos"),
    ("Huehuetenango", "Huehuetenango"),
    ("Chiché", "Quiché"),
    ("San Juan Ixcoy", "Huehuetenango"),
    ("Santa Eulalia", "Huehuetenango"),
    ("San Antonio Palopó", "Sololá"),
    ("Senahú", "Alta Verapaz"),
    ("Totonicapán", "Totonicapán"),
    ("La Esperanza", "Quetzaltenango"),
    ("Mazatenango", "Suchitepéquez"),
    ("San Rafael La Independencia", "Huehuetenango"),
    ("Chicamán", "Quiché"),
    ("El Quetzal", "San Marcos"),
    ("San Martín Sacatepéquez", "Quetzaltenango"),
    ("Ignorado", "Desconocido"),
    ("El Tumbador", "San Marcos"),
    ("Santo Domingo Suchitepéquez", "Suchitepéquez"),
    ("Chahal", "Alta Verapaz"),
    ("Zacualpa", "Quiché"),
    ("Fray Bartolomé de Las Casas", "Alta Verapaz"),
    ("Oratorio", "Santa Rosa"),
    ("Villa Canales", "Guatemala"),
    ("Retalhuleu", "Retalhuleu"),
    ("Almolonga", "Quetzaltenango"),
    ("Guastatoya", "El Progreso"),
    ("Chuarrancho", "Guatemala"),
    ("Atescatempa", "Jutiapa"),
    ("Chinautla", "Guatemala"),
    ("San Juan Chamelco", "Alta Verapaz"),
    ("Tamahú", "Alta Verapaz"),
    ("San Sebastián Huehuetenango", "Huehuetenango"),
    ("Cajolá", "Quetzaltenango"),
    ("Flores", "Petén"),
    ("San Pablo", "San Marcos"),
    ("Tecpán Guatemala", "Chimaltenango"),
    ("Iztapa", "Escuintla"),
    ("La Libertad", "Petén"),
    ("Tiquisate", "Escuintla"),
    ("Esquipulas", "Chiquimula"),
    ("Pachalum", "Quiché"),
    ("Malacatán", "San Marcos"),
    ("San Lorenzo", "San Marcos"),
    ("San Miguel Siguilá", "Quetzaltenango"),
    ("Nahualá", "Sololá"),
    ("Santa María de Jesús", "Sacatepéquez"),
    ("Camotán", "Chiquimula"),
    ("Olopa", "Chiquimula"),
    ("San Vicente Pacaya", "Escuintla"),
    ("San José La Arada", "Chiquimula"),
    ("Pajapita", "San Marcos"),
    ("San Pedro Jocopilas", "Quiché"),
    ("Fraijanes", "Guatemala"),
    ("Palencia", "Guatemala"),
    ("Santa Catarina Pinula", "Guatemala"),
    ("Magdalena Milpas Altas", "Sacatepéquez"),
    ("Siquinalá", "Escuintla"),
    ("Santa Clara La Laguna", "Sololá"),
    ("El Tejar", "Chimaltenango"),
    ("San José Poaquil", "Chimaltenango"),
    ("Sibilia", "Quetzaltenango"),
    ("Casillas", "Santa Rosa"),
    ("Panajachel", "Sololá"),
    ("San Andrés Itzapa", "Chimaltenango"),
    ("Salcajá", "Quetzaltenango"),
    ("Coatepeque", "Quetzaltenango"),
    ("Taxisco", "Santa Rosa"),
    ("El Palmar", "Quetzaltenango"),
    ("San Francisco Zapotitlán", "Suchitepéquez"),
    ("Chiquimulilla", "Santa Rosa"),
    ("Jalpatagua", "Jutiapa"),
    ("San Bernardino", "Suchitepéquez"),
    ("Santa Catarina Ixtahuacán", "Sololá"),
    ("Santa Lucía Cotzumalguapa", "Escuintla"),
    ("San Pedro Sacatepéquez", "Guatemala"),
    ("San Marcos", "San Marcos"),
    ("Todos Santos Cuchumatán", "Huehuetenango"),
    ("San Pedro Necta", "Huehuetenango"),
    ("Guanagazapa", "Escuintla"),
    ("Colotenango", "Huehuetenango"),
    ("Canillá", "Quiché"),
    ("Sipacapa", "San Marcos"),
    ("Santa Ana Huista", "Huehuetenango"),
    ("Joyabaj", "Quiché"),
    ("San Antonio Ilotenango", "Quiché"),
    ("Sibinal", "San Marcos"),
    ("San Andrés Sajcabajá", "Quiché"),
    ("San Pedro Carchá", "Alta Verapaz"),
    ("Tactic", "Alta Verapaz"),
    ("Cobán", "Alta Verapaz"),
    ("San Cristóbal Verapaz", "Alta Verapaz"),
    ("Chisec", "Alta Verapaz"),
    ("San Antonio Suchitepéquez", "Suchitepéquez"),
    ("San Andrés Villa Seca", "Suchitepéquez"),
    ("Sayaxché", "Petén"),
    ("El Estor", "Izabal"),
    ("Zunilito", "Suchitepéquez"),
    ("Concepción Chiquirichapa", "Quetzaltenango"),
    ("Cuyotenango", "Suchitepéquez"),
    ("Zaragoza", "Chimaltenango"),
    ("Santa Lucía Milpas Altas", "Sacatepéquez"),
    ("San Miguel Acatán", "Huehuetenango"),
    ("San Agustín Acasaguastlán", "El Progreso"),
    ("Jacaltenango", "Huehuetenango"),
    ("Raxruhá", "Alta Verapaz"),
    ("Santa Rosa de Lima", "Santa Rosa"),
    ("Chicacao", "Suchitepéquez"),
    ("Livingston", "Izabal"),
    ("Nueva Santa Rosa", "Santa Rosa"),
    ("Purulhá", "Baja Verapaz"),
    ("Sumpango", "Sacatepéquez"),
    ("San Luis Jilotepeque", "Jalapa"),
    ("San Juan La Laguna", "Sololá"),
    ("Pueblo Nuevo", "Suchitepéquez"),
    ("Tejutla", "San Marcos"),
    ("Las cruces", "Petén"),
    ("San Gabriel", "Suchitepéquez"),
    ("La Democracia", "Huehuetenango"),
    ("El Progreso", "Jutiapa"),
    ("Santiago Chimaltenango", "Huehuetenango"),
    ("San Francisco", "Petén"),
    ("San Andrés", "Petén"),
    ("Quesada", "Jutiapa"),
    ("San Pedro Ayampuc", "Guatemala"),
    ("Parramos", "Chimaltenango"),
    ("Barberena", "Santa Rosa"),
    ("San Martín Jilotepeque", "Chimaltenango"),
    ("San Andrés Xecul", "Totonicapán"),
    ("Catarina", "San Marcos"),
    ("Champerico", "Retalhuleu"),
    ("San Antonio Aguas Calientes", "Sacatepéquez"),
    ("Zacapa", "Zacapa"),
    ("Santiago Atitlán", "Sololá"),
    ("Morales", "Izabal"),
    ("Granados", "Baja Verapaz"),
    ("Asunción Mita", "Jutiapa"),
    ("La Blanca", "San Marcos"),
    ("Mataquescuintla", "Jalapa"),
    ("Cantel", "Quetzaltenango"),
    ("San José Ojetenam", "San Marcos"),
    ("San Felipe", "Retalhuleu"),
    ("Esquipulas Palo Gordo", "San Marcos"),
    ("Santa Cruz Verapaz", "Alta Verapaz"),
    ("Santa Catarina Barahona", "Sacatepéquez"),
    ("La Gomera", "Escuintla"),
    ("Santa Bárbara", "Huehuetenango"),
    ("San Gaspar Ixchil", "Huehuetenango"),
    ("Cuilco", "Huehuetenango"),
    ("San José El Ídolo", "Suchitepéquez"),
    ("Pueblo Nuevo", "San Marcos"),
    ("San Rafael Pie de La Cuesta", "San Marcos"),
    ("Patzún", "Chimaltenango"),
    ("Concepción Huista", "Huehuetenango"),
    ("Usumatlán", "Zacapa"),
    ("Santa Catarina Palopó", "Sololá"),
    ("Huitán", "Quetzaltenango"),
    ("Palestina de Los Altos", "Quetzaltenango"),
    ("Salamá", "Baja Verapaz"),
    ("Ayutla", "San Marcos"),
    ("Acatenango", "Chimaltenango"),
    ("Santa María Chiquimula", "Totonicapán"),
    ("Chiantla", "Huehuetenango"),
    ("San Juan Bautista", "Suchitepéquez"),
    ("Ciudad Vieja", "Sacatepéquez"),
    ("GUATEMALA", "Guatemala"),
    ("CHINAUTLA", "Guatemala"),
    ("SAN PEDRO AYAMPUC", "Guatemala"),
    ("FRAIJANES", "Guatemala"),
    ("SAN JOSÉ PINULAC", "Guatemala"),
    ("MIXCO", "Guatemala"),
    ("VILLA NUEVA", "Guatemala"),
    ("VILLA CANALES", "Guatemala"),
    ("SANTA CATARINA PINULA", "Guatemala"),
    ("SAN JOSÉ DEL GOLFO", "Guatemala"),
    ("PALENCIA", "Guatemala"),
    ("SAN JUAN SACATEPÉQUEZ", "Guatemala"),
    ("SAN RAIMUNDO", "Guatemala"),
    ("AMATITLÁN", "Guatemala"),
    ("PETAPA", "Guatemala"),
    ("ESCUINTLA", "Escuintla"),
    ("SAN PEDRO SACATEPÉQUEZ", "Guatemala"),
    ("JUTIAPA", "Jutiapa"),
    ("SANTA CRUZ DEL QUICHÉ", "Quiché"),
    ("SAN MIGUEL IXTAHUACÁN", "Huehuetenango"),
    ("SAN PEDRO PINULA", "Jalapa"),
    ("SANTA LUCÍA COTZUMALGUAPA", "Escuintla"),
    ("CHUARRANCHO", "Guatemala"),
    ("PALÍN", "Escuintla"),
    ("ANTIGUA GUATEMALA", "Sacatepéquez"),
    ("SAN JOSÉ", "Escuintla"),
    ("BARBERENA", "Santa Rosa"),
    ("JALAPA", "Jalapa"),
    ("IGNORADO", "Desconocido"),
    ("SAN LUIS JILITEPEQUE", "Jalapa"),
    ("TECPÁN GUATEMALA", "Chimaltenango"),
    ("GUASTATOYA", "El Progreso"),
    ("SAN LUCAS SACATEPÉQUEZ", "Sacatepéquez"),
    ("CABRICÁN", "Quetzaltenango"),
    ("NUEVA SANTA ROSA", "Santa Rosa"),
    ("MAZATENANGO", "Suchitepéquez"),
    ("PUERTO BARRIOS", "Izabal"),
    ("MORAZÁN", "El Progreso"),
    ("LIVINGSTON", "Izabal"),
    ("ZARAGOZA", "Chimaltenango"),
    ("CHIMALTENANGO", "Chimaltenango"),
    ("SAN JOSÉ POAQUIL", "Chimaltenango"),
    ("SAN MARTÍN JILOTEPEQUE", "Chimaltenango"),
    ("COMALAPA", "Chimaltenango"),
    ("PATZICÍA", "Chimaltenango"),
    ("SANTA APOLONIA", "Chimaltenango"),
    ("PATZÚN", "Chimaltenango"),
    ("POCHUTA", "Chimaltenango"),
    ("ACATENANGO", "Chimaltenango"),
    ("YEPOCAPA", "Chimaltenango"),
    ("SAN ANDRÉS ITZAPA", "Chimaltenango"),
    ("PARRAMOS", "Chimaltenango"),
    ("EL TEJAR", "Chimaltenango"),
    ("SACAPULAS", "Quiché"),
    ("SANTA CRUZ BALANYÁ", "Chimaltenango"),
    ("SANTA MARÍA DE JESÚS", "Sacatepéquez"),
    ("SAN FRANCISCO EL ALTO", "Totonicapán"),
    ("SUMPANGO", "Sacatepéquez"),
    ("LA DEMOCRACIA", "Escuintla"),
    ("SIQUINALÁ", "Escuintla"),
    ("MASAGUA", "Escuintla"),
    ("TIQUISATE", "Escuintla"),
    ("LA GOMERA", "Escuintla"),
    ("GUANAGAZAPA", "Escuintla"),
    ("IZTAPA", "Escuintla"),
    ("SAN VICENTE PACAYA", "Escuintla"),
    ("NUEVA CONCEPCIÓN", "Escuintla"),
    ("NUEVO PROGRESO", "San Marcos"),
    ("SANTA ROSA DE LIMA", "Santa Rosa"),
    ("CUILAPA", "Santa Rosa"),
    ("GUAZACAPÁN", "Santa Rosa"),
    ("ORATORIO", "Santa Rosa"),
    ("PUEBLO NUEVO VIÑAS", "Santa Rosa"),
    ("SANTA MARÍA IXHUATÁN", "Santa Rosa"),
    ("SANTA CRUZ NARANJO", "Santa Rosa"),
    ("CASILLAS", "Santa Rosa"),
    ("SAN RAFAEL LAS FLORES", "Santa Rosa"),
    ("CHIQUIMULILLA", "Santa Rosa"),
    ("TAXISCO", "Santa Rosa"),
    ("SAN JUAN TECUACO", "Santa Rosa"),
    ("MATAQUESCUINTLA", "Jalapa"),
    ("SAN JOSÉ CHACAYÁ", "Sololá"),
    ("NAHUALÁ", "Sololá"),
    ("SANTA CATARINA IXTAHUACÁN", "Sololá"),
    ("CONCEPCIÓN", "Sololá"),
    ("SAN ANDRÉS SEMETABAJ", "Sololá"),
    ("SANTA CATARINA PALOPÓ", "Sololá"),
    ("SANTA CRUZ LA LAGUNA", "Sololá"),
    ("SAN JUAN LA LAGUNA", "Sololá"),
    ("SANTIAGO ATITLÁN", "Sololá"),
    ("SANTA CLARA LA LAGUNA", "Sololá"),
    ("SAN LUCAS TOLIMÁN", "Sololá"),
    ("SOLOLÁ", "Sololá"),
    ("SANTA LUCÍA UTATLÁN", "Sololá"),
    ("PANAJACHEL", "Sololá"),
    ("SAN ANTONIO PALOPÓ", "Sololá"),
    ("SAN PEDRO LA LAGUNA", "Sololá"),
    ("SAN PABLO LA LAGUNA", "Sololá"),
    ("SAN MARCOS LA LAGUNA", "Sololá"),
    ("TOTONICAPÁN", "Totonicapán"),
    ("SAN ANDRÉS XECUL", "Totonicapán"),
    ("SAN CRISTÓBAL TOTONICAPÁN", "Totonicapán"),
    ("MONOSTENANGO", "Totonicapán"),
    ("SAN BARTOLO", "Totonicapán"),
    ("SAN LUCÍA LA REFORMA", "Totonicapán"),
    ("SANTA MARÍA CHIQUIMULA", "Totonicapán"),
    ("QUETZALTENANGO", "Quetzaltenango"),
    ("SAN FRANCISCO ZAPOTITLÁN", "Quetzaltenango"),
    ("CAJOLÁ", "Quetzaltenango"),
    ("SAN CARLOS SIJA", "Quetzaltenango"),
    ("SAN MATEO", "Quetzaltenango"),
    ("SALCAJÁ", "Quetzaltenango"),
    ("OLINTEPEQUE", "Quetzaltenango"),
    ("SIBILIA", "Quetzaltenango"),
    ("OSTUNCALCO", "Quetzaltenango"),
    ("CONCEPCIÓN CHIQUIRICHAPA", "Quetzaltenango"),
    ("SAN MARTÍN SACATEPÉQUEZ", "Quetzaltenango"),
    ("ALMOLONGA", "Quetzaltenango"),
    ("CANTEL", "Quetzaltenango"),
    ("ZUNIL", "Quetzaltenango"),
    ("HUITÁN", "Quetzaltenango"),
    ("COLOMBA", "Quetzaltenango"),
    ("EL PALMAR", "Quetzaltenango"),
    ("COATEPEQUE", "Quetzaltenango"),
    ("GÉNOVA", "Quetzaltenango"),
    ("FLORES COSTA CUCA", "Quetzaltenango"),
    ("PAJAPITA", "San Marcos"),
    ("LA ESPERANZA", "Quetzaltenango"),
    ("EL QUETZAL", "San Marcos"),
    ("PALESTINA DE LOS ALTOS", "Quetzaltenango"),
    ("SAN MIGUEL SIGÜILÁ", "Quetzaltenango"),
    ("SAN ANTONIO SACATEPÉQUEZ", "Sacatepéquez"),
    ("SAN JUAN COTZAL", "Quiché"),
    ("SAN FRANCISCO LA UNIÓN", "Quiché"),
    ("CUYOTENANGO", "Suchitepéquez"),
    ("SAN SEBASTIÁN", "Quetzaltenango"),
    ("CHAPERICO", "Retalhuleu"),
    ("RETALHULEU", "Retalhuleu"),
    ("NUEVO SAN CARLOS", "Retalhuleu"),
    ("SAN ANDRÉS VILLA SECA", "Suchitepéquez"),
    ("SAN MARTÍN ZAPOTITLÁN", "Retalhuleu"),
    ("SAN FELIPE", "Retalhuleu"),
    ("EL ASINTAL", "Retalhuleu"),
    ("SANTA CRUZ MULUÁ", "Retalhuleu"),
    ("HUEHUETENANGO", "Huehuetenango"),
    ("SAN MARCOS", "San Marcos"),
    ("SAN RAFAEL PIÉ DE LA CUESTA", "San Marcos"),
    ("COMITANCILLO", "San Marcos"),
    ("CONCEPCIÓN TUTUAPA", "San Marcos"),
    ("TACANÁ", "San Marcos"),
    ("SIBINAL", "San Marcos"),
    ("TAJUMULCO", "San Marcos"),
    ("TEJUTLA", "San Marcos"),
    ("EL TUMBADOR", "San Marcos"),
    ("EL RODEO", "San Marcos"),
    ("MALACATÁN", "San Marcos"),
    ("OCÓS", "San Marcos"),
    ("SAN PABLO", "San Marcos"),
    ("LA REFORMA", "San Marcos"),
    ("IXCHIGUÁN", "San Marcos"),
    ("SIPACAPA", "San Marcos"),
    ("ESQUIPULAS PALO GORDO", "San Marcos"),
    ("RÍO BLANCO", "San Marcos"),
    ("SAN LORENZO", "San Marcos"),
    ("LA BLANCA", "San Marcos"),
    ("CATARINA", "San Marcos"),
    ("SAN JOSÉ OJETENAM", "San Marcos"),
    ("SAN CRISTÓBAL CUCHO", "San Marcos"),
    ("AYUTLA", "San Marcos"),
    ("CHIANTLA", "Huehuetenango"),
    ("SANTA BÁRBARA", "Huehuetenango"),
    ("AGUACATÁN", "Huehuetenango"),
    ("LA DEMOCRACIA", "Huehuetenango"),
    ("COLOTENANGO", "Huehuetenango"),
    ("SAN SEBASTIÁN HUEHUETENANGO", "Huehuetenango"),
    ("TECTITÁN", "Huehuetenango"),
    ("MALACATANCITO", "Huehuetenango"),
    ("CUILCO", "Huehuetenango"),
    ("SAN PEDRO NECTA", "Huehuetenango"),
    ("SOLOMA", "Huehuetenango"),
    ("LA LIBERTAD", "Huehuetenango"),
    ("SAN MIGUEL ACATÁN", "Huehuetenango"),
    ("SAN RAFAEL LA INDEPENDENCIA", "Huehuetenango"),
    ("TODOS SANTOS CUCHUMATÁN", "Huehuetenango"),
    ("BARILLAS", "Huehuetenango"),
    ("SANTA EULALIA", "Huehuetenango"),
    ("SAN JUAN IXCOY", "Huehuetenango"),
    ("SAN ANTONIO HUISTA", "Huehuetenango"),
    ("SAN SEBASTIÁN COATÁN", "Huehuetenango"),
    ("SAN GASPAR IXCHIL", "Huehuetenango"),
    ("SAN JUAN ATITÁN", "Huehuetenango"),
    ("NENTÓN", "Huehuetenango"),
    ("JACALTENANGO", "Huehuetenango"),
    ("IXTAHUCÁN", "Huehuetenango"),
    ("SAN RAFAEL PETZAL", "Huehuetenango"),
    ("PETATAN", "Huehuetenango"),
    ("SANTIAGO CHIMALTENANGO", "Huehuetenango"),
    ("SANTA ANA HUISTA", "Huehuetenango"),
    ("CONCEPCIÓN HUISTA", "Huehuetenango"),
    ("IXCÁN", "Quiché"),
    ("ZACUALPA", "Quiché"),
    ("CHAJUL", "Quiché"),
    ("CHICHICASTENANGO", "Quiché"),
    ("PATZITÉ", "Quiché"),
    ("SAN ANTONIO ILOTENANGO", "Quiché"),
    ("CUNÉN", "Quiché"),
    ("JOYABAJ", "Quiché"),
    ("NEBAJ", "Quiché"),
    ("SAN ANDRÉS SAJCABAJÁ", "Quiché"),
    ("SAN BARTALOMÉ JOCOTENANGO", "Quiché"),
    ("CANILLÁ", "Quiché"),
    ("CHICAMÁN", "Quiché"),
    ("PACHALUM", "Quiché"),
    ("CHICHÉ", "Quiché"),
    ("CHINIQUE", "Quiché"),
    ("COBÁN", "Alta Verapaz"),
    ("USPANTÁN", "Quiché"),
    ("SALAMÁ", "Baja Verapaz"),
    ("SAN MIGUEL CHICAJ", "Baja Verapaz"),
    ("RABINAL", "Baja Verapaz"),
    ("CUBULCO", "Baja Verapaz"),
    ("GRANADOS", "Baja Verapaz"),
    ("SAN JERÓNIMO", "Baja Verapaz"),
    ("PURULHÁ", "Baja Verapaz"),
    ("EL CHOL", "Baja Verapaz"),
    ("SAN PEDRO CARCHÁ", "Alta Verapaz"),
    ("SAN JUAN CHAMELCO", "Alta Verapaz"),
    ("SANTA CRUZ VERAPAZ", "Alta Verapaz"),
    ("SAN CRISTÓBAL VERAPAZ", "Alta Verapaz"),
    ("TACTIC", "Alta Verapaz"),
    ("TAMAHÚ", "Alta Verapaz"),
    ("TUCURÚ", "Alta Verapaz"),
    ("PANZÓS", "Alta Verapaz"),
    ("SENAHÚ", "Alta Verapaz"),
    ("LANQUÍN", "Alta Verapaz"),
    ("CHISEC", "Alta Verapaz"),
    ("CHAHAL", "Alta Verapaz"),
    ("FRAY BARTOLOMÉ DE LAS CASAS", "Alta Verapaz"),
    ("SAN CATALINA LA TINTA", "Alta Verapaz"),
    ("RAXRUJÀ", "Alta Verapaz"),
    ("CAHABÓN", "Alta Verapaz"),
    ("SAYAXCHÉ", "Petén"),
    ("SAN LUIS", "Petén"),
    ("FLORES", "Petén"),
    ("SAN BENITO", "Petén"),
    ("SAN ANDRÉS", "Petén"),
    ("SAN FRANCISCO", "Petén"),
    ("DOLORES", "Petén"),
    ("EL CHAL", "Petén"),
    ("MELCHOR DE MENCOS", "Petén"),
    ("POPTÚN", "Petén"),
    ("LAS CRUCES", "Petén"),
    ("SANTA ANA", "Petén"),
    ("GUALÁN", "Zacapa"),
    ("ASUNCIÓN MITA", "Jutiapa"),
    ("SAN ANTONIO LA PAZ", "El Progreso"),
    ("EL JÍCARO", "El Progreso"),
    ("SAN AGUSTÍN ACASAGUASTLAN", "El Progreso"),
    ("SANARATE", "El Progreso"),
    ("SAN CRISTÓBAL ACASAGUASTLAN", "El Progreso"),
    ("SANSARE", "El Progreso"),
    ("AGUA BLANCA", "Jutiapa"),
    ("JOCOTENANGO", "Sacatepéquez"),
    ("PASTORES", "Sacatepéquez"),
    ("SANTO DOMINGO XENACOJ", "Sacatepéquez"),
    ("SANTIAGO SACATEPÉQUEZ", "Sacatepéquez"),
    ("SAN BARTOLOMÉ MILPAS ALTAS", "Sacatepéquez"),
    ("SANTA LUCÍA MILPAS ALTAS", "Sacatepéquez"),
    ("MAGDALENA MILPAS ALTAS", "Sacatepéquez"),
    ("CIUDAD VIEJA", "Sacatepéquez"),
    ("SAN MIGUEL DUEÑAS", "Sacatepéquez"),
    ("ALOTENANGO", "Sacatepéquez"),
    ("SANTA CATARINA BARAHONA", "Sacatepéquez"),
    ("SAMAYAC", "Suchitepéquez"),
    ("SANTO DOMINGO SUCHITEPÉQUEZ", "Suchitepéquez"),
    ("SAN ANTONIO SUCHITEPÉQUEZ", "Suchitepéquez"),
    ("ZUNILITO", "Suchitepéquez"),
    ("SAN JOSÉ LA MÁQUINA", "Suchitepéquez"),
    ("SAN BERNARDINO", "Suchitepéquez"),
    ("SAN JOSÉ EL ÍDOLO", "Suchitepéquez"),
    ("SAN PABLO JOCOPILAS", "Suchitepéquez"),
    ("SAN GABRIEL", "Suchitepéquez"),
    ("CHICACAO", "Suchitepéquez"),
    ("SAN JUAN BAUTISTA", "Suchitepéquez"),
    ("PATULUL", "Suchitepéquez"),
    ("SANTO TOMÁS LA UNIÓN", "Suchitepéquez"),
    ("PUEBLO NUEVO", "Suchitepéquez"),
    ("RÍO BRAVO", "Suchitepéquez"),
    ("MORALES", "Izabal"),
    ("EL ESTOR", "Izabal"),
    ("ZACAPA", "Zacapa"),
    ("TECULUTÁN", "Zacapa"),
    ("SAN JORGE", "Zacapa"),
    ("ESTANZUELA", "Zacapa"),
    ("RÍO HONDO", "Zacapa"),
    ("HUITÉ", "Zacapa"),
    ("ESQUIPULAS", "Chiquimula"),
    ("SAN JACINTO", "Chiquimula"),
    ("CHIQUIMULA", "Chiquimula"),
    ("OLOPA", "Chiquimula"),
    ("SAN JUAN ERMITA", "Chiquimula"),
    ("JOCOTÁN", "Chiquimula"),
    ("CONCEPCIÓN LAS MINAS", "Chiquimula"),
    ("QUETZALTEPEQUE", "Chiquimula"),
    ("IPALA", "Chiquimula"),
    ("MONJAS", "Jalapa"),
    ("SAN MANUEL CHAPARRÓN", "Jalapa"),
    ("CONGUACO", "Jutiapa"),
    ("SANTA CATARINA MITA", "Jutiapa"),
    ("YUPILTEPEQUE", "Jutiapa"),
    ("COMAPA", "Jutiapa"),
    ("JALPATAGUA", "Jutiapa"),
    ("EL PROGRESO", "Jutiapa"),
    ("ATESCATEMPA", "Jutiapa"),
    ("JEREZ", "Jutiapa"),
    ("ZAPOTITLÁN", "Jutiapa"),
    ("PASACO", "Jutiapa"),
    ("QUEZADA", "Jutiapa"),
    ("SAN MIGUEL PANÁN", "Jutiapa"),
    ("LOS AMATES", "Izabal"),
    ("USUMATLÁN", "Zacapa"),
    ("SAN DIEGO", "Zacapa"),
    ("LA UNIÓN", "Zacapa"),
    ("SAN JOSÉ LA ARADA", "Chiquimula"),
    ("EL ADELANTO", "Jutiapa"),
    ("SAN JOSÉ ACATEMPA", "Jutiapa"),
    ("SAN ANTONIO AGUAS CALIENTES", "Sacatepéquez"),
    ("CABAÑAS", "Zacapa"),
    ("CAMOTÁN", "Chiquimula"),
    ("MOYUTA", "Jutiapa"),
    ("SAN MATEO IXTATÁN", "Huehuetenango"),
    ("La Tinta", "Alta Verapaz"),
    ("Fray Bartolomé de las Casas", "Alta Verapaz"),
    ("Guanazapa", "Escuintla"),
    ("San Joseé La Máquina", "Suchitepéquez"),
    ("San Cristobal Acasaguastlán", "El Progreso"),
    ("Zapotitlán", "Jutiapa"),
    ("San Francisco el Alto", "Totonicapán"),
    ("Santa Lucia Utatlán", "Sololá"),
    ("San Antonio la Paz", "El Progreso"),
    ("San Francisco la Unión", "Quiché"),
    ("Santiago Sactepéquez", "Sacatepéquez"),
    ("El Jícaro", "El Progreso"),
    ("Santa Catalina Ixtahuacán", "Sololá"),
    ("Santo Tomás la Unión", "Suchitepéquez"),
    ("San Andres Semetabaj", "Sololá"),
    ("Melchor", "Petén"),
    ("San Rafael la Independencia", "Huehuetenango"),
    ("Santiago Chimaltenanango", "Huehuetenango"),
    ("Poptún", "Petén"),
    ("San Luis", "Petén"),
    ("Comitancillo", "San Marcos"),
    ("Dolores", "Petén"),
    ("San Antonio Sacatepéquez", "San Marcos"),
    ("Nuevo San Carlos", "Retalhuleu"),
    ("San Miguel Ixtahuacán", "San Marcos"),
    ("San Lucas Sacatepéquez", "Sacatepéquez"),
    ("Rabinal", "Baja Verapaz"),
    ("San Miguel Chicaj", "Baja Verapaz"),
    ("San José el Golfo", "Guatemala"),
    ("San Raimundo", "Guatemala"),
    ("Petapa", "Guatemala"),
    ("Guatemala", "Guatemala"),
    ("Alotenango", "Sacatepéquez"),
    ("Chimaltenango", "Chimaltenango"),
    ("Yepocapa", "Chimaltenango"),
    ("San Rafael las Flores", "Santa Rosa"),
    ("Santiago Atitlán", "Sololá"),
    ("San Cristobal Totonicapán", "Totonicapán"),
    ("Santa Lucía La Reforma", "Totonicapán"),
    ("Salcajá", "Quetzaltenango"),
    ("Olintepeque", "Quetzaltenango"),
    ("Ostuncalco", "Quetzaltenango"),
    ("Colomba", "Quetzaltenango"),
    ("Palestina de los Altos", "Quetzaltenango"),
    ("San José El Idolo", "Suchitepéquez"),
    ("Patulul", "Suchitepéquez"),
    ("Santo Tomás La Unión", "Suchitepéquez"),
    ("Rio Bravo", "Escuintla"),
    ("San Rafael Pié de La Cuesta", "San Marcos"),
    ("San Marcos", "San Marcos"),
    ("El Rodeo", "San Marcos"),
    ("Ocós", "San Marcos"),
    ("Ixtahuacán", "Huehuetenango"),
    ("San Antonio Huista", "Huehuetenango"),
    ("Chichicastenango", "Quiché"),
    ("Uspantán", "Quiché"),
    ("Ixcán", "Quiché"),
    ("Nebaj", "Quiché"),
    ("Tucurú", "Alta Verapaz"),
    ("Lanquín", "Alta Verapaz"),
    ("Cahabón", "Alta Verapaz"),
    ("Santa Catalina la Tinta", "Alta Verapaz"),
    ("Jocotán", "Chiquimula"),
    ("Quetzaltepeque", "Chiquimula"),
    ("San José Acatempa", "Jutiapa"),
    ("Villa Nueva", "Guatemala"),
    ("Comalapa", "Chimaltenango"),
    ("Pochuta", "Chimaltenango"),
    ("Santa Lucía Cotzumalguapa", "Escuintla"),
    ("San Bartolo", "Totonicapán"),
    ("Quetzaltenango", "Quetzaltenango"),
    ("Mazatenango", "Suchitepéquez"),
    ("El Asintal", "Retalhuleu"),
    ("Rio Blanco", "San Marcos"),
    ("Barillas", "Huehuetenango"),
    ("Santa Cruz del Quiché", "Quiché"),
    ("El Chol", "Baja Verapaz"),
    ("Chiquimula", "Chiquimula"),
    ("San Juan Erminta", "Chiquimula"),
    ("San Agustín Acasaguastlán", "El Progreso"),
    ("Totonicapán", "Totonicapán"),
    ("San Francisco La Unión", "Quiché"),
    ("Coatepeque", "Quetzaltenango"),
    ("Unión Cantinil", "Huehuetenango"),
    ("Chinique", "Quiché"),
    ("Antigua Guatemala", "Sacatepéquez"),
    ("Amatitlán", "Guatemala"),
    ("San Antonio Sacatepéquez", "San Marcos"),
    ("Huehuetenango", "Huehuetenango"),
    ("Cobán", "Alta Verapaz"),
    ("Jalapa", "Jalapa"),
    ("Nueva Concepción", "Escuintla"),
    ("San Francisco El Alto", "Totonicapán"),
    ("Soloma", "Huehuetenango"),
    ("Río Hondo", "Zacapa"),
    ("Jutiapa", "Jutiapa"),
    ("Jerez", "Jutiapa"),
    ("Malacatán", "San Marcos"),
    ("San Lucas Toliman", "Sololá"),
    ("San Mateo Ixtatán", "Huehuetenango"),
    ("Tamahú", "Alta Verapaz"),
    ("Flores", "Petén"),
    ("Camotán", "Chiquimula"),
    ("San José Chacayá", "Sololá"),
    ("Cabricán", "Quetzaltenango"),
    ("El Tumbador", "San Marcos"),
    ("San Juan Sacatepéquez", "Guatemala"),
    ("Villa Canales", "Guatemala"),
    ("Cajolá", "Quetzaltenango"),
    ("La Esperanza", "Quetzaltenango"),
    ("El Quetzal", "San Marcos"),
    ("Sibinal", "San Marcos"),
    ("Patzún", "Chimaltenango"),
    ("Parramos", "Chimaltenango"),
    ("Flores Costa Cuca", "Quetzaltenango"),
    ("Cuilapa", "Santa Rosa"),
    ("Colotenango", "Huehuetenango"),
    ("Chajul", "Quiché"),
    ("El Estor", "Izabal"),
    ("Mixco", "Guatemala"),
    ("San Andrés Itzapa", "Chimaltenango"),
    ("Palín", "Escuintla"),
    ("Casillas", "Santa Rosa"),
    ("Pajapita", "San Marcos"),
    ("San Andrés Xecul", "Totonicapán"),
    ("La Democracia", "Escuintla"),
    ("Sololá", "Sololá"),
    ("Esquipulas", "Chiquimula"),
    ("San Carlos Sija", "Quetzaltenango"),
    ("Chicamán", "Quiché"),
    ("Ayutla", "San Marcos"),
    ("Salamá", "Baja Verapaz"),
    ("Concepción Chiquirichapa", "Quetzaltenango"),
    ("Tecpán Guatemala", "Chimaltenango"),
    ("San Antonio Ilotenango", "Quiché"),
    ("Momostenango", "Totonicapán"),
    ("San Pedro Necta", "Huehuetenango"),
    ("Pastores", "Sacatepéquez"),
    ("Cantel", "Quetzaltenango"),
    ("Chiché", "Quiché"),
    ("Siquinalá", "Escuintla"),
    ("Chuarrancho", "Guatemala"),
    ("San Bernardino", "Suchitepéquez"),
    ("Rabinal", "Baja Verapaz"),
    ("San Martín Zapotitlán", "Retalhuleu"),
    ("Santa Catarina Pinula", "Guatemala"),
    ("Chicacao", "Suchitepéquez"),
    ("San Cristóbal Verapaz", "Alta Verapaz"),
    ("San José Poaquil", "Chimaltenango"),
    ("San Vicente Pacaya", "Escuintla"),
    ("San Andrés Sajcabajá", "Quiché"),
    ("San Antonio Suchitepéquez", "Suchitepéquez"),
    ("Taxisco", "Santa Rosa"),
    ("Los Amates", "Izabal"),
    ("Palencia", "Guatemala"),
    ("Fraijanes", "Guatemala"),
    ("San Antonio La Paz", "El Progreso"),
    ("Santo Domingo Xenacoj", "Sacatepéquez"),
    ("Patzicía", "Chimaltenango"),
    ("La Gomera", "Escuintla"),
    ("San José", "Escuintla"),
    ("Santa María Chiquimula", "Totonicapán"),
    ("Sibilia", "Quetzaltenango"),
    ("San Miguel Siguilá", "Quetzaltenango"),
    ("San Francisco Zapotitlán", "Suchitepéquez"),
    ("Santo Domingo Suchitepéquez", "Suchitepéquez"),
    ("Samayac", "Suchitepéquez"),
    ("San Pablo Jocopilas", "Suchitepéquez"),
    ("Santa Bárbara", "Huehuetenango"),
    ("Retalhuleu", "Retalhuleu"),
    ("San Pedro Sacatepéquez", "Guatemala"),
    ("Concepción Tutuapa", "San Marcos"),
    ("Tejutla", "San Marcos"),
    ("Catarina", "San Marcos"),
    ("San Lorenzo", "San Marcos"),
    ("San Miguel Acatán", "Huehuetenango"),
    ("San Juan Atitán", "Huehuetenango"),
    ("Aguacatán", "Huehuetenango"),
    ("San Rafael Petzal", "Huehuetenango"),
    ("Santiago Chimaltenango", "Huehuetenango"),
    ("Santa Ana Huista", "Huehuetenango"),
    ("Zacualpa", "Quiché"),
    ("San Pedro Jocopilas", "Quiché"),
    ("Cunén", "Quiché"),
    ("Joyabaj", "Quiché"),
    ("Sacapulas", "Quiché"),
    ("Canillá", "Quiché"),
    ("Purulhá", "Baja Verapaz"),
    ("Tactic", "Alta Verapaz"),
    ("San Pedro Carchá", "Alta Verapaz"),
    ("Chisec", "Alta Verapaz"),
    ("Chahal", "Alta Verapaz"),
    ("La Unión", "Zacapa"),
    ("Olopa", "Chiquimula"),
    ("San Jacinto", "Chiquimula"),
    ("Asunción Mita", "Jutiapa"),
    ("Chinautla", "Guatemala"),
    ("San Martín Sacatepéquez", "Quetzaltenango"),
    ("Guastatoya", "El Progreso"),
    ("Santa Eulalia", "Huehuetenango"),
    ("Santa Catarina Ixtahuacán", "Sololá"),
    ("San Andrés Semetabaj", "Sololá"),
    ("San Juan Bautista", "Suchitepéquez"),
    ("Pueblo Nuevo", "Suchitepéquez"),
    ("Cuilco", "Huehuetenango"),
    ("San Sebastián Huehuetenango", "Huehuetenango"),
    ("Moyuta", "Jutiapa"),
    ("Sayaxché", "Petén"),
    ("Ipala", "Chiquimula"),
    ("San Juan Cotzal", "Quiché"),
    ("La Libertad", "Petén"),
    ("Las cruces", "Petén"),
    ("Sumpango", "Sacatepéquez"),
    ("Escuintla", "Escuintla"),
    ("San Mateo", "Quetzaltenango"),
    ("Mataquescuintla", "Jalapa"),
    ("San Pedro Ayampuc", "Guatemala"),
    ("Zaragoza", "Chimaltenango"),
    ("Panzós", "Alta Verapaz"),
    ("San Francisco", "Petén"),
    ("San Martín Jilotepeque", "Chimaltenango"),
    ("Pueblo Nuevo Viñas", "Santa Rosa"),
    ("Sipacapa", "San Marcos"),
    ("San Sebastián", "Quetzaltenango"),
    ("Patzité", "Quiché"),
    ("Santa Cruz Verapaz", "Alta Verapaz"),
    ("San Benito", "Petén"),
    ("San Miguel Panán", "Jutiapa"),
    ("San Juan Chamelco", "Alta Verapaz"),
    ("Raxruhá", "Alta Verapaz"),
    ("Almolonga", "Quetzaltenango"),
    ("San Rafael La Independencia", "Huehuetenango"),
    ("Nueva Santa Rosa", "Santa Rosa"),
    ("San Miguel Chicaj", "Baja Verapaz"),
    ("Pasaco", "Jutiapa"),
    ("San Gabriel", "Suchitepéquez"),
    ("Cubulco", "Baja Verapaz"),
    ("Santa Lucía Milpas Altas", "Sacatepéquez"),
    ("Granados", "Baja Verapaz"),
    ("Ciudad Vieja", "Sacatepéquez"),
    ("San Miguel Dueñas", "Sacatepéquez"),
    ("Santiago Sacatepéquez", "Sacatepéquez"),
    ("San Antonio Palopó", "Sololá"),
    ("Santa Cruz Balanyá", "Chimaltenango"),
    ("Poptún", "Petén"),
    ("Santa Cruz Naranjo", "Santa Rosa"),
    ("San José El Ídolo", "Suchitepéquez"),
    ("Pueblo Nuevo", "San Marcos"),
    ("San Rafael Pie de La Cuesta", "San Marcos"),
    ("San Pablo", "San Marcos"),
    ("Usumatlán", "Zacapa"),
    ("Malacatancito", "Huehuetenango"),
    ("Magdalena Milpas Altas", "Sacatepéquez"),
    ("Acatenango", "Chimaltenango"),
    ("Concepción Las Minas", "Chiquimula"),
    ("Yupiltepeque", "Jutiapa"),
    ("Santa Catarina Barahona", "Sacatepéquez"),
    ("Quesada", "Jutiapa"),
    ("El Palmar", "Quetzaltenango"),
    ("El Tejar", "Chimaltenango"),
    ("Fray Bartolomé de Las Casas", "Alta Verapaz"),
    ("Concepción", "Sololá"),
    ("Comitancillo", "San Marcos"),
    ("La Reforma", "San Marcos"),
    ("Nentón", "Huehuetenango"),
    ("San Antonio Aguas Calientes", "Sacatepéquez"),
    ("Atescatempa", "Jutiapa"),
    ("GUATEMALA", "Guatemala"),
    ("VILLA NUEVA", "Guatemala"),
    ("SANTA CATARINA PINULA", "Guatemala"),
    ("SAN JOSÉ PINULAC", "Guatemala"),
    ("SAN JOSÉ DEL GOLFO", "Guatemala"),
    ("PALENCIA", "Guatemala"),
    ("CHINAUTLA", "Guatemala"),
    ("SAN PEDRO AYAMPUC", "Guatemala"),
    ("MIXCO", "Guatemala"),
    ("SAN JUAN SACATEPÉQUEZ", "Guatemala"),
    ("SAN RAIMUNDO", "Guatemala"),
    ("FRAIJANES", "Guatemala"),
    ("AMATITLÁN", "Guatemala"),
    ("VILLA CANALES", "Guatemala"),
    ("PETAPA", "Guatemala"),
    ("CHUARRANCHO", "Guatemala"),
    ("SAN PEDRO SACATEPÉQUEZ", "Guatemala"),
    ("CHIMALTENANGO", "Chimaltenango"),
    ("SAN JOSÉ POAQUIL", "Chimaltenango"),
    ("SAN MARTÍN JILOTEPEQUE", "Chimaltenango"),
    ("COMALAPA", "Chimaltenango"),
    ("SANTA APOLONIA", "Chimaltenango"),
    ("TECPÁN GUATEMALA", "Chimaltenango"),
    ("PATZÚN", "Chimaltenango"),
    ("POCHUTA", "Chimaltenango"),
    ("PATZICÍA", "Chimaltenango"),
    ("ACATENANGO", "Chimaltenango"),
    ("YEPOCAPA", "Chimaltenango"),
    ("SAN ANDRÉS ITZAPA", "Chimaltenango"),
    ("PARRAMOS", "Chimaltenango"),
    ("ZARAGOZA", "Chimaltenango"),
    ("EL TEJAR", "Chimaltenango"),
    ("SANTA CRUZ BALANYÁ", "Chimaltenango"),
    ("SAN MARTÍN SACATEPÉQUEZ", "Quetzaltenango"),
    ("ESCUINTLA", "Escuintla"),
    ("SANTA LUCÍA COTZUMALGUAPA", "Escuintla"),
    ("LA DEMOCRACIA", "Escuintla"),
    ("SIQUINALÁ", "Escuintla"),
    ("MASAGUA", "Escuintla"),
    ("TIQUISATE", "Escuintla"),
    ("LA GOMERA", "Escuintla"),
    ("GUANAGAZAPA", "Escuintla"),
    ("SAN JOSÉ", "Escuintla"),
    ("IZTAPA", "Escuintla"),
    ("PALÍN", "Escuintla"),
    ("SAN VICENTE PACAYA", "Escuintla"),
    ("NUEVA CONCEPCIÓN", "Escuintla"),
    ("CUILAPA", "Santa Rosa"),
    ("BARBERENA", "Santa Rosa"),
    ("SANTA ROSA DE LIMA", "Santa Rosa"),
    ("CASILLAS", "Santa Rosa"),
    ("SAN RAFAEL LAS FLORES", "Santa Rosa"),
    ("ORATORIO", "Santa Rosa"),
    ("CHIQUIMULILLA", "Santa Rosa"),
    ("SANTA MARÍA IXHUATÁN", "Santa Rosa"),
    ("SANTA CRUZ NARANJO", "Santa Rosa"),
    ("PUEBLO NUEVO VIÑAS", "Santa Rosa"),
    ("NUEVA SANTA ROSA", "Santa Rosa"),
    ("TAXISCO", "Santa Rosa"),
    ("GUAZACAPÁN", "Santa Rosa"),
    ("SAN JOSÉ CHACAYÁ", "Sololá"),
    ("NAHUALÁ", "Sololá"),
    ("SANTA CATARINA IXTAHUCÁN", "Sololá"),
    ("CONCEPCIÓN", "Sololá"),
    ("SAN ANDRÉS SEMETABAJ", "Sololá"),
    ("SANTA CATARINA PALOPÓ", "Sololá"),
    ("SANTA CRUZ LA LAGUNA", "Sololá"),
    ("SAN JUAN LA LAGUNA", "Sololá"),
    ("SANTIAGO ATITLÁN", "Sololá"),
    ("SANTA CLARA LA LAGUNA", "Sololá"),
    ("SAN LUCAS TOLIMÁN", "Sololá"),
    ("SOLOLÁ", "Sololá"),
    ("SANTA LUCÍA UTATLÁN", "Sololá"),
    ("PANAJACHEL", "Sololá"),
    ("SAN ANTONIO PALOPÓ", "Sololá"),
    ("SAN PEDRO LA LAGUNA", "Sololá"),
    ("SAN PABLO LA LAGUNA", "Sololá"),
    ("SAN MARCOS LA LAGUNA", "Sololá"),
    ("TOTONICAPÁN", "Totonicapán"),
    ("SAN CRISTÓBAL TOTONICAPÁN", "Totonicapán"),
    ("SAN FRANCISCO EL ALTO", "Totonicapán"),
    ("MONOSTENANGO", "Totonicapán"),
    ("SAN BARTOLO", "Totonicapán"),
    ("SAN ANDRÉS XECUL", "Totonicapán"),
    ("SAN LUCÍA LA REFORMA", "Totonicapán"),
    ("SANTA MARÍA CHIQUIMULA", "Totonicapán"),
    ("QUETZALTENANGO", "Quetzaltenango"),
    ("SALCAJÁ", "Quetzaltenango"),
    ("OLINTEPEQUE", "Quetzaltenango"),
    ("SIBILIA", "Quetzaltenango"),
    ("CABRICÁN", "Quetzaltenango"),
    ("OSTUNCALCO", "Quetzaltenango"),
    ("SAN MATEO", "Quetzaltenango"),
    ("CONCEPCIÓN CHIQUIRICHAPA", "Quetzaltenango"),
    ("ALMOLONGA", "Quetzaltenango"),
    ("CANTEL", "Quetzaltenango"),
    ("HUITÁN", "Quetzaltenango"),
    ("COLOMBA", "Quetzaltenango"),
    ("EL PALMAR", "Quetzaltenango"),
    ("COATEPEQUE", "Quetzaltenango"),
    ("GÉNOVA", "Quetzaltenango"),
    ("FLORES COSTA CUCA", "Quetzaltenango"),
    ("LA ESPERANZA", "Quetzaltenango"),
    ("CAJOLÁ", "Quetzaltenango"),
    ("PALESTINA DE LOS ALTOS", "Quetzaltenango"),
    ("SAN MIGUEL SIGÜILÁ", "Quetzaltenango"),
    ("SAN FRANCISCO LA UNIÓN", "Quiché"),
    ("SAN CARLOS SIJA", "Quetzaltenango"),
    ("RETALHULEU", "Retalhuleu"),
    ("SAN SEBASTIÁN", "Quetzaltenango"),
    ("SAN MARTÍN ZAPOTITLÁN", "Retalhuleu"),
    ("SAN FELIPE", "Retalhuleu"),
    ("SAN ANDRÉS VILLA SECA", "Suchitepéquez"),
    ("CHAPERICO", "Retalhuleu"),
    ("NUEVO SAN CARLOS", "Retalhuleu"),
    ("EL ASINTAL", "Retalhuleu"),
    ("SANTA CRUZ MULUÁ", "Retalhuleu"),
    ("SAN MARCOS", "San Marcos"),
    ("SAN ANTONIO SACATEPÉQUEZ", "San Marcos"),
    ("COMITANCILLO", "San Marcos"),
    ("SAN MIGUEL IXTAHUACÁN", "San Marcos"),
    ("CONCEPCIÓN TUTUAPA", "San Marcos"),
    ("TACANÁ", "San Marcos"),
    ("SIBINAL", "San Marcos"),
    ("TAJUMULCO", "San Marcos"),
    ("TEJUTLA", "San Marcos"),
    ("SAN RAFAEL PIÉ DE LA CUESTA", "San Marcos"),
    ("EL TUMBADOR", "San Marcos"),
    ("EL RODEO", "San Marcos"),
    ("MALACATÁN", "San Marcos"),
    ("OCÓS", "San Marcos"),
    ("SAN PABLO", "San Marcos"),
    ("EL QUETZAL", "San Marcos"),
    ("LA REFORMA", "San Marcos"),
    ("PAJAPITA", "San Marcos"),
    ("IXCHIGUÁN", "San Marcos"),
    ("SIPACAPA", "San Marcos"),
    ("ESQUIPULAS PALO GORDO", "San Marcos"),
    ("RÍO BLANCO", "San Marcos"),
    ("SAN LORENZO", "San Marcos"),
    ("LA BLANCA", "San Marcos"),
    ("NUEVO PROGRESO", "San Marcos"),
    ("AYUTLA", "San Marcos"),
    ("CATARINA", "San Marcos"),
    ("SAN JOSÉ OJETENAM", "San Marcos"),
    ("SAN CRISTÓBAL CUCHO", "San Marcos"),
    ("HUEHUETENANGO", "Huehuetenango"),
    ("MALACATANCITO", "Huehuetenango"),
    ("CUILCO", "Huehuetenango"),
    ("SAN PEDRO NECTA", "Huehuetenango"),
    ("SOLOMA", "Huehuetenango"),
    ("SANTA BÁRBARA", "Huehuetenango"),
    ("LA LIBERTAD", "Huehuetenango"),
    ("SAN MIGUEL ACATÁN", "Huehuetenango"),
    ("SAN RAFAEL LA INDEPENDENCIA", "Huehuetenango"),
    ("TODOS SANTOS CUCHUMATÁN", "Huehuetenango"),
    ("SANTA EULALIA", "Huehuetenango"),
    ("TECTITÁN", "Huehuetenango"),
    ("SAN JUAN IXCOY", "Huehuetenango"),
    ("SAN ANTONIO HUISTA", "Huehuetenango"),
    ("SAN SEBASTIÁN COATÁN", "Huehuetenango"),
    ("AGUACATÁN", "Huehuetenango"),
    ("SAN GASPAR IXCHIL", "Huehuetenango"),
    ("SAN JUAN ATITÁN", "Huehuetenango"),
    ("NENTÓN", "Huehuetenango"),
    ("JACALTENANGO", "Huehuetenango"),
    ("IXTAHUCÁN", "Huehuetenango"),
    ("SAN RAFAEL PETZAL", "Huehuetenango"),
    ("PETATAN", "Huehuetenango"),
    ("SAN SEBASTIÁN HUEHUETENANGO", "Huehuetenango"),
    ("SANTIAGO CHIMALTENANGO", "Huehuetenango"),
    ("SANTA ANA HUISTA", "Huehuetenango"),
    ("BARILLAS", "Huehuetenango"),
    ("CONCEPCIÓN HUISTA", "Huehuetenango"),
    ("SANTA CRUZ DEL QUICHÉ", "Quiché"),
    ("ZACUALPA", "Quiché"),
    ("CHAJUL", "Quiché"),
    ("CHICHICASTENANGO", "Quiché"),
    ("PATZITÉ", "Quiché"),
    ("SAN ANTONIO ILOTENANGO", "Quiché"),
    ("SAN PEDRO JOCOPILAS", "Quiché"),
    ("CUNÉN", "Quiché"),
    ("SAN JUAN COTZAL", "Quiché"),
    ("JOYABAJ", "Quiché"),
    ("NEBAJ", "Quiché"),
    ("SAN ANDRÉS SAJCABAJÁ", "Quiché"),
    ("SAN BARTALOMÉ JOCOTENANGO", "Quiché"),
    ("CANILLÁ", "Quiché"),
    ("CHICAMÁN", "Quiché"),
    ("IXCÁN", "Quiché"),
    ("PACHALUM", "Quiché"),
    ("CHICHÉ", "Quiché"),
    ("CHINIQUE", "Quiché"),
    ("USPANTÁN", "Quiché"),
    ("SACAPULAS", "Quiché"),
    ("SALAMÁ", "Baja Verapaz"),
    ("SAN MIGUEL CHICAJ", "Baja Verapaz"),
    ("RABINAL", "Baja Verapaz"),
    ("CUBULCO", "Baja Verapaz"),
    ("GRANADOS", "Baja Verapaz"),
    ("SAN JERÓNIMO", "Baja Verapaz"),
    ("PURULHÁ", "Baja Verapaz"),
    ("EL CHOL", "Baja Verapaz"),
    ("COBÁN", "Alta Verapaz"),
    ("SANTA CRUZ VERAPAZ", "Alta Verapaz"),
    ("SAN CRISTÓBAL VERAPAZ", "Alta Verapaz"),
    ("TACTIC", "Alta Verapaz"),
    ("TAMAHÚ", "Alta Verapaz"),
    ("TUCURÚ", "Alta Verapaz"),
    ("PANZÓS", "Alta Verapaz"),
    ("SENAHÚ", "Alta Verapaz"),
    ("SAN PEDRO CARCHÁ", "Alta Verapaz"),
    ("SAN JUAN CHAMELCO", "Alta Verapaz"),
    ("LANQUÍN", "Alta Verapaz"),
    ("CHISEC", "Alta Verapaz"),
    ("CHAHAL", "Alta Verapaz"),
    ("FRAY BARTOLOMÉ DE LAS CASAS", "Alta Verapaz"),
    ("SAN CATALINA LA TINTA", "Alta Verapaz"),
    ("RAXRUJÀ", "Alta Verapaz"),
    ("CAHABÓN", "Alta Verapaz"),
    ("FLORES", "Petén"),
    ("SAN ANDRÉS", "Petén"),
    ("SAN FRANCISCO", "Petén"),
    ("DOLORES", "Petén"),
    ("SAN LUIS", "Petén"),
    ("SAYAXCHÉ", "Petén"),
    ("MELCHOR DE MENCOS", "Petén"),
    ("POPTÚN", "Petén"),
    ("LAS CRUCES", "Petén"),
    ("EL CHAL", "Petén"),
    ("SAN BENITO", "Petén"),
    ("SANTA ANA", "Petén"),
    ("GUASTATOYA", "El Progreso"),
    ("MORAZÁN", "El Progreso"),
    ("SAN AGUSTÍN ACASAGUASTLAN", "El Progreso"),
    ("SAN CRISTÓBAL ACASAGUASTLAN", "El Progreso"),
    ("EL JÍCARO", "El Progreso"),
    ("SANSARE", "El Progreso"),
    ("SANARATE", "El Progreso"),
    ("SAN ANTONIO LA PAZ", "El Progreso"),
    ("ANTIGUA GUATEMALA", "Sacatepéquez"),
    ("JOCOTENANGO", "Sacatepéquez"),
    ("PASTORES", "Sacatepéquez"),
    ("SUMPANGO", "Sacatepéquez"),
    ("SANTO DOMINGO XENACOJ", "Sacatepéquez"),
    ("SANTIAGO SACATEPÉQUEZ", "Sacatepéquez"),
    ("SAN BARTOLOMÉ MILPAS ALTAS", "Sacatepéquez"),
    ("SAN LUCAS SACATEPÉQUEZ", "Sacatepéquez"),
    ("SANTA LUCÍA MILPAS ALTAS", "Sacatepéquez"),
    ("MAGDALENA MILPAS ALTAS", "Sacatepéquez"),
    ("SANTA MARÍA DE JESÚS", "Sacatepéquez"),
    ("CIUDAD VIEJA", "Sacatepéquez"),
    ("SAN MIGUEL DUEÑAS", "Sacatepéquez"),
    ("ALOTENANGO", "Sacatepéquez"),
    ("SANTA CATARINA BARAHONA", "Sacatepéquez"),
    ("MAZATENANGO", "Suchitepéquez"),
    ("CUYOTENANGO", "Suchitepéquez"),
    ("SAN FRANCISCO ZAPOTITLÁN", "Suchitepéquez"),
    ("SAN BERNARDINO", "Suchitepéquez"),
    ("SAN JOSÉ EL ÍDOLO", "Suchitepéquez"),
    ("SANTO DOMINGO SUCHITEPÉQUEZ", "Suchitepéquez"),
    ("SAMAYAC", "Suchitepéquez"),
    ("SAN PABLO JOCOPILAS", "Suchitepéquez"),
    ("SAN ANTONIO SUCHITEPÉQUEZ", "Suchitepéquez"),
    ("SAN GABRIEL", "Suchitepéquez"),
    ("CHICACAO", "Suchitepéquez"),
    ("SAN JUAN BAUTISTA", "Suchitepéquez"),
    ("SANTO TOMÁS LA UNIÓN", "Suchitepéquez"),
    ("ZUNILITO", "Suchitepéquez"),
    ("PUEBLO NUEVO", "Suchitepéquez"),
    ("RÍO BRAVO", "Suchitepéquez"),
    ("SAN JOSE LA MÁQUINA", "Suchitepéquez"),
    ("PUERTO BARRIOS", "Izabal"),
    ("LIVINGSTON", "Izabal"),
    ("EL ESTOR", "Izabal"),
    ("MORALES", "Izabal"),
    ("ZACAPA", "Zacapa"),
    ("RÍO HONDO", "Zacapa"),
    ("GUALÁN", "Zacapa"),
    ("TECULUTÁN", "Zacapa"),
    ("SAN DIEGO", "Zacapa"),
    ("HUITÉ", "Zacapa"),
    ("CHIQUIMULA", "Chiquimula"),
    ("SAN JUAN ERMITA", "Chiquimula"),
    ("ESQUIPULAS", "Chiquimula"),
    ("CONCEPCIÓN LAS MINAS", "Chiquimula"),
    ("QUETZALTEPEQUE", "Chiquimula"),
    ("SAN JACINTO", "Chiquimula"),
    ("IPALA", "Chiquimula"),
    ("JALAPA", "Jalapa"),
    ("SAN PEDRO PINULA", "Jalapa"),
    ("SAN MANUEL CHAPARRÓN", "Jalapa"),
    ("MONJAS", "Jalapa"),
    ("MATAQUESCUINTLA", "Jalapa"),
    ("JUTIAPA", "Jutiapa"),
    ("EL PROGRESO", "Jutiapa"),
    ("SANTA CATARINA MITA", "Jutiapa"),
    ("AGUA BLANCA", "Jutiapa"),
    ("ASUNCIÓN MITA", "Jutiapa"),
    ("YUPILTEPEQUE", "Jutiapa"),
    ("ATESCATEMPA", "Jutiapa"),
    ("JEREZ", "Jutiapa"),
    ("ZAPOTITLÁN", "Jutiapa"),
    ("JALPATAGUA", "Jutiapa"),
    ("PASACO", "Jutiapa"),
    ("QUEZADA", "Jutiapa"),
    ("SAN ANTONIO AGUAS CALIENTES", "Sacatepéquez"),
    ("SAN MIGUEL PANÁN", "Jutiapa"),
    ("LOS AMATES", "Izabal"),
    ("USUMATLÁN", "Zacapa"),
    ("LA UNIÓN", "Zacapa"),
    ("OLOPA", "Chiquimula"),
    ("EL ADELANTO", "Jutiapa"),
    ("COMAPA", "Jutiapa"),
    ("SAN JOSÉ ACATEMPA", "Jutiapa"),
    ("PATULUL", "Suchitepéquez"),
    ("CABAÑAS", "Zacapa"),
    ("SAN JOSÉ LA ARADA", "Chiquimula"),
    ("SAN LUIS JILITEPEQUE", "Jalapa"),
    ("MOYUTA", "Jutiapa"),
    ("CONGUACO", "Jutiapa"),
    ("ESTANZUELA", "Zacapa"),
    ("SAN JORGE", "Zacapa"),
    ("JOCOTÁN", "Chiquimula"),
    ("CAMOTÁN", "Chiquimula"),
    ("LA DEMOCRACIA", "Huehuetenango"),
    ("CHIANTLA", "Huehuetenango"),
    ("SAN MATEO IXTATÁN", "Huehuetenango"),
    ("ZUNIL", "Quetzaltenango"),
    ("La Tinta", "Alta Verapaz"),
    ("Fray Bartolomé de las Casas", "Alta Verapaz"),
    ("Guanazapa", "Escuintla"),
    ("San Joseé La Máquina", "Suchitepéquez"),
    ("San Cristobal Acasaguastlán", "El Progreso"),
    ("Santa Lucia Utatlán", "Sololá"),
    ("San Antonio la Paz", "El Progreso"),
    ("San Francisco la Unión", "Quiché"),
    ("San Francisco el Alto", "Totonicapán"),
    ("Santiago Sactepéquez", "Sacatepéquez"),
    ("Santa Catalina Ixtahuacán", "Sololá"),
    ("Santo Tomás la Unión", "Suchitepéquez"),
    ("San Andres Semetabaj", "Sololá"),
    ("Melchor", "Petén"),
    ("San Rafael la Independencia", "Huehuetenango"),
    ("El Jícaro", "El Progreso"),
    ("Santiago Chimaltenanango", "Huehuetenango"),
    ("Génova", "Quetzaltenango"),
    ("San Luis", "Petén"),
    ("Sanarate", "El Progreso"),
    ("Pachalum", "Quiché"),
    ("Santa Apolonia", "Chimaltenango"),
    ("San José Ojetenam", "San Marcos"),
    ("Zacapa", "Zacapa"),
    ("San Cristóbal Cucho", "San Marcos"),
    ("Jalpatagua", "Jutiapa"),
    ("San Lucas Sacatepéquez", "Sacatepéquez"),
    ("Nuevo Progreso", "San Marcos"),
    ("San Bartolomé Milpas Altas", "Sacatepéquez"),
    ("Petatán", "Huehuetenango"),
    ("El Progreso", "Jutiapa"),
    ("LA DEMOCARACIA", "Escuintla"),
    ("Pueblo Nuevo", "Suchitepequez"),
    ("RAXUJÀ", "Alta Verapaz"),
    ("HUEHUETENAGO", "Huehuetenango"),
    ("Mazatenango", "Suchitepéquez"),
    ("Patulul", "Suchitepéquez"),
    ("San Bernardino", "Suchitepéquez"),
    ("San Miguel Panán", "Suchitepéquez"),
]

In [8]:
municipio_to_depto = dict(municipios_departamentos)

In [9]:
df['HEC_DEPTO'] = df['HEC_DEPTO'].fillna(df['HEC_DEPTOMCPIO'].map(municipio_to_depto))
df['DEPTO'] = df['DEPTO'].fillna(df['DEPTO_MCPIO'].map(municipio_to_depto))
# Por alguna razon pueblo nuevo es el único que no le funciona el map
df["HEC_DEPTO"] = df["HEC_DEPTO"].fillna("Suchitepéquez")
df["DEPTO"] = df["DEPTO"].fillna("Suchitepéquez")

In [10]:
df.isnull().sum().sort_values(ascending=False).iloc[0:30]

TIPO_DISCAQ                 510267
INST_DONDE_DENUNCIO         453640
AGR_DEDICA                  415448
CONDUCENTE                  397273
ORGANISMO_JURISDICCIONAL    397273
ORGANISMO_REMITE            397273
VIC_OCUP                    346326
TIPO_MEDIDA                 342847
LEY_APLICABLE               277144
MEDIDAS_SEGURIDAD           243237
VIC_DEDICA                  177488
AGR_OTROS_HOM               156826
AGRESORES_OTROS_TOTAL       156826
AGR_OTRAS_N_AS              156826
AGR_OTROS_N_OS              156826
AGR_OTRAS_MUJ               156826
AGR_OCUP                    108311
NUM_HIJ_HOM                   2382
TOTAL_HIJOS                   2382
VIC_EST_CIV                   2382
NUM_HIJ_MUJ                   2382
VIC_ALFAB                      972
VIC_ESCOLARIDAD                972
VIC_TRABAJA                    972
AGR_EST_CIV                     37
HEC_DEPTOMCPIO                   0
AGR_SEXO                         0
AGR_ALFAB                        0
AGR_ESCOLARIDAD     

In [11]:
df["ORGANISMO_REMITE"] = df["ORGANISMO_REMITE"].fillna("Ignorado")
df["INST_DONDE_DENUNCIO"] = df["INST_DONDE_DENUNCIO"].fillna("Ignorado")
df["AGR_DEDICA"] = df["AGR_DEDICA"].fillna("Ignorado")
df["CONDUCENTE"] = df["CONDUCENTE"].fillna("Ignorado")
df["ORGANISMO_JURISDICCIONAL"] = df["ORGANISMO_JURISDICCIONAL"].fillna(99.0)
df["VIC_OCUP"] = df["VIC_OCUP"].fillna("Ocupación ignorada")
df["TIPO_MEDIDA"] = df["TIPO_MEDIDA"].fillna("9999")
df["LEY_APLICABLE"] = df["LEY_APLICABLE"].fillna("IGNORADO")
df["MEDIDAS_SEGURIDAD"] = df["MEDIDAS_SEGURIDAD"].fillna("Ignorado")
df["VIC_DEDICA"] = df["VIC_DEDICA"].fillna("Ignorado")
df["AGRESORES_OTROS_TOTAL"] = df["AGRESORES_OTROS_TOTAL"].fillna(99.0)
df["AGR_OTROS_N_OS"] = df["AGR_OTROS_N_OS"].fillna(99.0)
df["AGR_OTROS_HOM"] = df["AGR_OTROS_HOM"].fillna(99.0)
df["AGR_OTRAS_MUJ"] = df["AGR_OTRAS_MUJ"].fillna(99.0)
df["AGR_OTRAS_N_AS"] = df["AGR_OTRAS_N_AS"].fillna(99.0)
df["AGR_OCUP"] = df["AGR_OCUP"].fillna("Ocupación ignorada")
df["VIC_EST_CIV"] = df["VIC_EST_CIV"].fillna("Ignorado")
df["TOTAL_HIJOS"] = df["TOTAL_HIJOS"].fillna(9999.0)
df["NUM_HIJ_HOM"] = df["NUM_HIJ_HOM"].fillna(9999.0)
df["NUM_HIJ_MUJ"] = df["NUM_HIJ_MUJ"].fillna(9999.0)
df["VIC_ALFAB"] = df["VIC_ALFAB"].fillna("Ignorado")
df["VIC_TRABAJA"] = df["VIC_TRABAJA"].fillna("Ignorado")
df["VIC_ESCOLARIDAD"] = df["VIC_ESCOLARIDAD"].fillna("Ignorado")
df["AGR_EST_CIV"] = df["AGR_EST_CIV"].fillna("Ignorado")

In [12]:
df.columns

Index(['AGR_ALFAB', 'AGR_DEDICA', 'AGR_EDAD', 'AGR_ESCOLARIDAD', 'AGR_EST_CIV',
       'AGR_GRUPET', 'AGR_NACIONAL', 'AGR_OCUP', 'AGR_SEXO', 'AGR_TRABAJA',
       'ANO_EMISION', 'DEPTO_MCPIO', 'DIA_EMISION', 'HEC_ANO', 'HEC_AREA',
       'HEC_DEPTOMCPIO', 'HEC_DIA', 'HEC_MES', 'HEC_RECUR_DENUN',
       'HEC_TIPAGRE', 'INST_DENUN_HECHO', 'INST_DONDE_DENUNCIO', 'MES_EMISION',
       'NUM_HIJ_HOM', 'NUM_HIJ_MUJ', 'OTRAS_VICTIMAS', 'QUIEN_REPORTA',
       'TIPO_DISCAQ', 'TOTAL_HIJOS', 'VIC_ALFAB', 'VIC_DEDICA', 'VIC_DISC',
       'VIC_EDAD', 'VIC_ESCOLARIDAD', 'VIC_EST_CIV', 'VIC_GRUPET',
       'VIC_NACIONAL', 'VIC_OCUP', 'VIC_OTRAS_HOM', 'VIC_OTRAS_MUJ',
       'VIC_OTRAS_N_AS', 'VIC_OTRAS_N_OS', 'VIC_REL_AGR', 'VIC_SEXO',
       'VIC_TRABAJA', 'DEPTO', 'HEC_DEPTO', 'AGR_OTROS_HOM',
       'ORGANISMO_REMITE', 'AGR_OTRAS_MUJ', 'AGR_OTROS_N_OS', 'CONDUCENTE',
       'LEY_APLICABLE', 'AGR_OTRAS_N_AS', 'AGRESORES_OTROS_TOTAL',
       'MEDIDAS_SEGURIDAD', 'TIPO_MEDIDA', 'ORGANISMO_JURISDICCIO

In [13]:
df["VIC_DISC"].value_counts()

VIC_DISC
Sin discapacidad    490189
Ignorado             27018
Con discapacidad      4748
Name: count, dtype: int64

In [14]:
df["TIPO_DISCAQ"] = df["TIPO_DISCAQ"].fillna(df["VIC_DISC"].map({
    "Sin discapacidad": "No aplica",
    "Ignorado": "Ignorado",
    "Con discapacidad": "Ignorado"
}))

## Transformaciones

### Convertir tipos de dato

#### Obtener las columnas que no son numéricas

In [15]:
non_numerical = list(df.select_dtypes(exclude=["number"]).columns)
non_numerical.extend(['AGR_OCUP', 'DEPTO_MCPIO', 'HEC_DEPTOMCPIO', 'VIC_OCUP', 'TIPO_MEDIDA']) # Estas por alguna razón no las toma en cuenta por lo que las agrego manualmente

#### Obtener las columnas que son numéricas

In [16]:
numerical = list(df.select_dtypes(include=["number"]).columns)

#### Convertir las columnas no numéricas a datos de tipo categóricos para que sea más sencillo de lidiar con
Esto permite que el tipo de dato no sea tan pesado al ser una categoría

In [17]:
for col in non_numerical:
    df[col] = df[col].astype('category')

### Limpiar las columnas no numéricas
Convertirlas a minúsculas, eliminar los acentos y eliminar los espacios si tienen espacios al principio o al final de manera que se normalicen.
Sin embargo, aquí se trabaja con batches ya que si se trabaja todo de una vez completamente se vuelve demasiado costoso para la máquina por lo que es necesario trabajarlo por batches para que el kernel no crashee.

In [18]:
batch_size = 10000
for i in range(0, len(non_numerical), batch_size):
    batch = non_numerical[i:i + batch_size]
    for col in batch:
        df[col] = df[col].apply(lambda x: unidecode(str(x).lower().strip()))

## Relacionar las ocupaciones de las victimas y los agresores con el registro del INE
Esto permite que se puedan agrupar las ocupaciones de las víctimas y los agresores de manera que sea más sencillo y se pueda generar nuevas variables y demás para el clustering

In [19]:
ocupaciones = pd.read_csv("./ocupaciones/ocupaciones.csv", sep="|")
ocupaciones["1"] = ocupaciones["1"].fillna("Directores ejecutivos, personal directivo de la administración pública y miembros del poder ejecutivo y de los cuerpos legislativos")

In [20]:
def normalize(text):
    if pd.isnull(text):
        return ""
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)  # remove punctuation
    return text.strip()
    
ocupaciones["normalized"] = ocupaciones["5"].apply(normalize)

vic_to_ocup_map = {}

choices = ocupaciones["normalized"].unique()

for vic_ocup in df["VIC_OCUP"].unique():
    match, score, _ = process.extractOne(vic_ocup, choices, scorer=fuzz.token_sort_ratio)
    if score >= 70:  # You can tune this threshold
        vic_to_ocup_map[vic_ocup] = match
    else:
        vic_to_ocup_map[vic_ocup] = None  # No good match

for vic_ocup in df["AGR_OCUP"].unique():
    match, score, _ = process.extractOne(vic_ocup, choices, scorer=fuzz.token_sort_ratio)
    if score >= 70:  # You can tune this threshold
        vic_to_ocup_map[vic_ocup] = match
    else:
        vic_to_ocup_map[vic_ocup] = None  # No good match
        
unmatched_vic_to_ocup_map = {}
for vic_ocup in [key for key in vic_to_ocup_map.keys() if vic_to_ocup_map[key]==None]:
    match, score, _ = process.extractOne(vic_ocup, choices, scorer=fuzz.token_sort_ratio)
    if score >= 60:  # You can tune this threshold
        unmatched_vic_to_ocup_map[vic_ocup] = match
    else:
        unmatched_vic_to_ocup_map[vic_ocup] = None  # No good match

vic_agr_ocup_ocup_map = {}
vic_agr_ocup_ocup_map.update(vic_to_ocup_map)
vic_agr_ocup_ocup_map.update(unmatched_vic_to_ocup_map)
vic_agr_ocup_ocup_map["ocupacion ignorada"] = None

In [21]:
# Estos son los que no quedan bien
vic_agr_ocup_ocup_map["preparadores de fibras"]="operadores de máquinas de preparación de fibras, hilado y devanado"
vic_agr_ocup_ocup_map["tejedores con telares o de tejidos de punto y afines"]="operador de máquina tejedora"
vic_agr_ocup_ocup_map["camareros y tabarneros"]="camareros de barra"
vic_agr_ocup_ocup_map["porteros y guardianes y afines"]="guardian de edificio"
vic_agr_ocup_ocup_map["constructores con tecnicas y materiales tradicionales"]="constructores de casas"
vic_agr_ocup_ocup_map["taquigrafos y mecanografos"]="operadores de máquinas de procesamiento de texto ymecanógrafos"
vic_agr_ocup_ocup_map["camareros y azafatas"]="camarero de bus"
vic_agr_ocup_ocup_map["operadores de maquinas para elaborar frutos humedos y secos y hortalizas"]='operadorempacador de frutas u hortalizas'
vic_agr_ocup_ocup_map["limpiabotas y otros trabajadores callejeros"]="limpiadores de vehículos ventanas ropa y otralimpieza a mano"
vic_agr_ocup_ocup_map["operadores de maquinas para moles cereales y especias"]="operador de molinos de cereales"
vic_agr_ocup_ocup_map["maestros de nivel medio de la ensenanza preescolar"]="maestros preescolares"
vic_agr_ocup_ocup_map["criadores de ganado y otros animales domesticos, productores de leche y sus"]='criadores deganado'
vic_agr_ocup_ocup_map["otros operadores de maquinas y montadores"]="moldeadores \nsoldadores \nchapistas\ncaldereros  montadores \nde"
vic_agr_ocup_ocup_map["preparadores de fibras"]="operadores de máquinas de preparación de fibras, hilado y devanado"
vic_agr_ocup_ocup_map["tejedores con telares o de tejidos de punto y afines"]="operador de máquina tejedora"
vic_agr_ocup_ocup_map["camareros y tabarneros"]="camareros de barra"
vic_agr_ocup_ocup_map["porteros y guardianes y afines"]="guardian de edificio"
vic_agr_ocup_ocup_map["constructores con tecnicas y materiales tradicionales"]="constructores de casas"
vic_agr_ocup_ocup_map["taquigrafos y mecanografos"]="operadores de máquinas de procesamiento de texto ymecanógrafos"
vic_agr_ocup_ocup_map["camareros y azafatas"]="camarero de bus"
vic_agr_ocup_ocup_map["operadores de maquinas para elaborar frutos humedos y secos y hortalizas"]='operadorempacador de frutas u hortalizas'
vic_agr_ocup_ocup_map["limpiabotas y otros trabajadores callejeros"]="limpiadores de vehículos ventanas ropa y otralimpieza a mano"
vic_agr_ocup_ocup_map["operadores de maquinas para moles cereales y especias"]="operador de molinos de cereales"
vic_agr_ocup_ocup_map["maestros de nivel medio de la ensenanza preescolar"]="maestros preescolares"
vic_agr_ocup_ocup_map["criadores de ganado y otros animales domesticos, productores de leche y sus"]='criadores deganado'
vic_agr_ocup_ocup_map["otros operadores de maquinas y montadores"]="moldeadores \nsoldadores \nchapistas\ncaldereros  montadores \nde"
vic_agr_ocup_ocup_map["compradores"]='agente de compras'
vic_agr_ocup_ocup_map["sacerdotes de distintas religiones"]='sacerdote cristiano'
vic_agr_ocup_ocup_map["frutero (vendedor ambulante)"]='vendedor ambulante de frutas y verduras'
vic_agr_ocup_ocup_map["profesionales de la informatica, no clasificados bajo otros epígrafes"]='técnico en redesinformáticas'
vic_agr_ocup_ocup_map["odontologos"]="dentista"
vic_agr_ocup_ocup_map["operadores de entrada de datos"]='digitador de base de datos'
vic_agr_ocup_ocup_map["operadores de instalaciones de tratamiento quimico termico"]="operadores  de \ninstalaciones y  máquinas  de  productos  químicos  y"
vic_agr_ocup_ocup_map["agricultores y trabajadores calificados de plantaciones de arboles y arbustos"]='agricultores  y  trabajadores  calificados  de  plantaciones  de  árboles  y'
vic_agr_ocup_ocup_map["cesteros, bruceros y afines"]='vendedor ambulante de productos de consumo no comestibles'
vic_agr_ocup_ocup_map["escultores, pintores y afines"]="escultor"
vic_agr_ocup_ocup_map["matematicos y afines"]='matemático'
vic_agr_ocup_ocup_map["tecnicos en control de equipos informaticos"]='técnico en mantenimiento de equiposelectrónicos'
vic_agr_ocup_ocup_map["farmacologos, patologos y afines"]='farmacéutico y farmacólogo'
vic_agr_ocup_ocup_map["otros maestros e instructores de nivel medio"]='profesores de enseñanza secundaria'
vic_agr_ocup_ocup_map["guias"]='guía turístico'
vic_agr_ocup_ocup_map["criadores de especies acuaticas"]='criadores  y  trabajadores  pecuarios  calificados  de la  cría  de  animales'
vic_agr_ocup_ocup_map["operadores de calculadoras"]="matemático"
vic_agr_ocup_ocup_map["payasos, prestidigitadores, acrobatas y fines"]="payaso"
vic_agr_ocup_ocup_map["tasadores y subastadores"]="tasadores"
vic_agr_ocup_ocup_map["pintores, decoradores de vidrio, ceramica y otros materiales"]='pintor decorador de cerámica'
vic_agr_ocup_ocup_map["peones de montaje"]='ayudante de montaje manual'
vic_agr_ocup_ocup_map["peleteros y afines"]="peletero"
vic_agr_ocup_ocup_map["higienistas y otro personal sanitario"]='profesionales de la salud y la higiene laboral yambiental'
vic_agr_ocup_ocup_map["juez de alzada"]="juez de paz"
vic_agr_ocup_ocup_map["inspectores de edificios y de prevencion e investigacion de incendios"]='investigador deincendios'
vic_agr_ocup_ocup_map["operadores de maquinas para fabricar productos farmaceuticos y cosmeticos"]='operador demáquina de fabricación de productos farmacéuticos'
vic_agr_ocup_ocup_map["operadores de instalaciones de tratamiento termico de metales"]='controlador de instalacionesde tratamiento térmico de metales'
vic_agr_ocup_ocup_map["empleados de archivos"]='empleado de información'
vic_agr_ocup_ocup_map["empleados del servicio de personal"]="trabajadores de los servicios personales"
vic_agr_ocup_ocup_map["empleados de bibliotecas"]="bibliotecario"
vic_agr_ocup_ocup_map["directores de politicas y planificacion"]='especialistas en políticas de administración'
vic_agr_ocup_ocup_map["musicos, cantantes y bailarines callejeros, de cabaret y afines"]='bailarín de cabaret'
vic_agr_ocup_ocup_map["apicultores y sericicultores y trabajadores calificados de la apicultura y la se"]="apicultor"
vic_agr_ocup_ocup_map["operadores de carretillas elevadoras"]='operadores de grúas aparatos elevadores y afines'
vic_agr_ocup_ocup_map["operadores de instalaciones de tratamientos quimicos, no clasificados bajo otros"]='controlador de instalaciones de tratamiento químico térmico'
vic_agr_ocup_ocup_map["operadores de maquinas para fabricar municiones y explosivos"]='comandante de armada'
vic_agr_ocup_ocup_map["compradores"]='agente de compras'
vic_agr_ocup_ocup_map["sacerdotes de distintas religiones"]='sacerdote cristiano'
vic_agr_ocup_ocup_map["frutero (vendedor ambulante)"]='vendedor ambulante de frutas y verduras'
vic_agr_ocup_ocup_map["profesionales de la informatica, no clasificados bajo otros epígrafes"]='técnico en redesinformáticas'
vic_agr_ocup_ocup_map["odontologos"]="dentista"
vic_agr_ocup_ocup_map["operadores de entrada de datos"]='digitador de base de datos'
vic_agr_ocup_ocup_map["operadores de instalaciones de tratamiento quimico termico"]="operadores  de \ninstalaciones y  máquinas  de  productos  químicos  y"
vic_agr_ocup_ocup_map["agricultores y trabajadores calificados de plantaciones de arboles y arbustos"]='agricultores  y  trabajadores  calificados  de  plantaciones  de  árboles  y'
vic_agr_ocup_ocup_map["cesteros, bruceros y afines"]='vendedor ambulante de productos de consumo no comestibles'
vic_agr_ocup_ocup_map["escultores, pintores y afines"]="escultor"
vic_agr_ocup_ocup_map["matematicos y afines"]='matemático'
vic_agr_ocup_ocup_map["tecnicos en control de equipos informaticos"]='técnico en mantenimiento de equiposelectrónicos',
vic_agr_ocup_ocup_map["farmacologos, patologos y afines"]='farmacéutico y farmacólogo'
vic_agr_ocup_ocup_map["otros maestros e instructores de nivel medio"]='profesores de enseñanza secundaria'
vic_agr_ocup_ocup_map["guias"]='guía turístico'
vic_agr_ocup_ocup_map["criadores de especies acuaticas"]='criadores  y  trabajadores  pecuarios  calificados  de la  cría  de  animales',
vic_agr_ocup_ocup_map["operadores de calculadoras"]="matemático"
vic_agr_ocup_ocup_map["payasos, prestidigitadores, acrobatas y fines"]="payaso"
vic_agr_ocup_ocup_map["tasadores y subastadores"]="tasadores"
vic_agr_ocup_ocup_map["pintores, decoradores de vidrio, ceramica y otros materiales"]='pintor decorador de cerámica'
vic_agr_ocup_ocup_map["peones de montaje"]='ayudante de montaje manual'
vic_agr_ocup_ocup_map["peleteros y afines"]="peletero"
vic_agr_ocup_ocup_map["higienistas y otro personal sanitario"]='profesionales de la salud y la higiene laboral yambiental'
vic_agr_ocup_ocup_map["juez de alzada"]="juez de paz"
vic_agr_ocup_ocup_map["inspectores de edificios y de prevencion e investigacion de incendios"]='investigador deincendios'
vic_agr_ocup_ocup_map["operadores de maquinas para fabricar productos farmaceuticos y cosmeticos"]='operador demáquina de fabricación de productos farmacéuticos'
vic_agr_ocup_ocup_map["operadores de instalaciones de tratamiento termico de metales"]='controlador de instalacionesde tratamiento térmico de metales'
vic_agr_ocup_ocup_map["empleados de archivos"]='empleado de información'
vic_agr_ocup_ocup_map["empleados del servicio de personal"]="trabajadores de los servicios personales"
vic_agr_ocup_ocup_map["empleados de bibliotecas"]="bibliotecario"
vic_agr_ocup_ocup_map["directores de politicas y planificacion"]='especialistas en políticas de administración'
vic_agr_ocup_ocup_map["musicos, cantantes y bailarines callejeros, de cabaret y afines"]='bailarín de cabaret'
vic_agr_ocup_ocup_map["apicultores y sericicultores y trabajadores calificados de la apicultura y la se"]="apicultor"
vic_agr_ocup_ocup_map["operadores de carretillas elevadoras"]='operadores de grúas aparatos elevadores y afines'
vic_agr_ocup_ocup_map["operadores de instalaciones de tratamientos quimicos, no clasificados bajo otros"]='controlador de instalaciones de tratamiento químico térmico'
vic_agr_ocup_ocup_map["operadores de maquinas para fabricar municiones y explosivos"]='comandante de armada'
vic_agr_ocup_ocup_map['dirigentes y administradores de organizaciones de empleadores, de trabajadores y'] = 'directores administradores y comerciales'
vic_agr_ocup_ocup_map['dirigentes y administradores de organizaciones humanitarias y de otras organizaciones'] = 'directores administradores y comerciales'

In [22]:
ocupaciones = ocupaciones.rename({"0":"grupo", "1": "sub_grupo_principal", "2":"sub_grupo", "3":"grupo_unitario", "normalized":"ocupacion_normalizada", "5":"ocupacion"}, axis=1)

In [23]:
df["VIC_OCUP_INE"] = df["VIC_OCUP"].map(vic_agr_ocup_ocup_map)
df["AGR_OCUP_INE"] = df["AGR_OCUP"].map(vic_agr_ocup_ocup_map)

In [24]:
df['VIC_DEDICA'].value_counts()

VIC_DEDICA
quehaceres del hogar              321717
ignorado                          183320
estudiar                           12266
vive de sus renta o jubilacion      3179
desempleado                         1108
otro                                 345
alcoholico/drogadicto                 20
Name: count, dtype: int64

In [25]:
print(df['VIC_DEDICA'].value_counts())
print(df['VIC_OCUP'].value_counts())

VIC_DEDICA
quehaceres del hogar              321717
ignorado                          183320
estudiar                           12266
vive de sus renta o jubilacion      3179
desempleado                         1108
otro                                 345
alcoholico/drogadicto                 20
Name: count, dtype: int64
VIC_OCUP
ocupacion ignorada                                                                         352770
vendedores no clasificados bajo otros epigrafes                                             21232
limpiadores y asistentes domesticos                                                         14469
agricultores y trabajadores calificados de cultivos extensivos                              10717
maestros de ensenanza primaria                                                               8624
                                                                                            ...  
actores                                                                        

In [26]:
df['AGR_DEDICA'].value_counts()

AGR_DEDICA
ignorado                          442974
quehaceres del hogar               54645
desempleado                        11682
estudiar                            5759
vive de sus renta o jubilacion      4264
otro                                2163
alcoholico/drogadicto                468
Name: count, dtype: int64

In [27]:
df[df['AGR_OCUP_INE'].isna()][['AGR_OCUP']].value_counts()

AGR_OCUP                                      
ocupacion ignorada                                136423
ocupaciones no especificadas en las anteriores      8775
Name: count, dtype: int64

In [28]:

# Merge para VIC_OCUP_INE
df = df.merge(
    ocupaciones[[
        'ocupacion_normalizada', 
        'grupo', 
        'sub_grupo_principal', 
        'sub_grupo', 
        'grupo_unitario'
    ]].rename(columns={
        'grupo': 'VIC_OCUP_GRUPO',
        'sub_grupo_principal': 'VIC_OCUP_SUB_GRUPO_PRINCIPAL',
        'sub_grupo': 'VIC_OCUP_SUB_GRUPO',
        'grupo_unitario': 'VIC_OCUP_GRUPO_UNITARIO',
        'ocupacion_normalizada': 'VIC_OCUP_INE'
    }),
    on='VIC_OCUP_INE',
    how='left'
)

# Merge para AGR_OCUP_INE
df = df.merge(
    ocupaciones[[
        'ocupacion_normalizada', 
        'grupo', 
        'sub_grupo_principal', 
        'sub_grupo', 
        'grupo_unitario'
    ]].rename(columns={
        'grupo': 'AGR_OCUP_GRUPO',
        'sub_grupo_principal': 'AGR_OCUP_SUB_GRUPO_PRINCIPAL',
        'sub_grupo': 'AGR_OCUP_SUB_GRUPO',
        'grupo_unitario': 'AGR_OCUP_GRUPO_UNITARIO',
        'ocupacion_normalizada': 'AGR_OCUP_INE'
    }),
    on='AGR_OCUP_INE',
    how='left'
)

### Llenar los nulos de las ocupaciones

In [29]:
df["VIC_OCUP_INE"] = df["VIC_OCUP_INE"].fillna("Ocupación ignorada")
df["AGR_OCUP_INE"] = df["AGR_OCUP_INE"].fillna("Ocupación ignorada")
df["VIC_OCUP_GRUPO"] = df["VIC_OCUP_GRUPO"].fillna("Grupo ignorado")
df["AGR_OCUP_GRUPO"] = df["AGR_OCUP_GRUPO"].fillna("Grupo ignorado")
df["VIC_OCUP_SUB_GRUPO_PRINCIPAL"] = df["VIC_OCUP_SUB_GRUPO_PRINCIPAL"].fillna("Grupo ignorado")
df["AGR_OCUP_SUB_GRUPO_PRINCIPAL"] = df["AGR_OCUP_SUB_GRUPO_PRINCIPAL"].fillna("Grupo ignorado")
df["VIC_OCUP_SUB_GRUPO"] = df["VIC_OCUP_SUB_GRUPO"].fillna("Grupo ignorado")
df["AGR_OCUP_SUB_GRUPO"] = df["AGR_OCUP_SUB_GRUPO"].fillna("Grupo ignorado")
df["VIC_OCUP_GRUPO_UNITARIO"] = df["VIC_OCUP_GRUPO_UNITARIO"].fillna("Grupo ignorado")
df["AGR_OCUP_GRUPO_UNITARIO"] = df["AGR_OCUP_GRUPO_UNITARIO"].fillna("Grupo ignorado")

In [30]:
df["VIC_OCUP_GRUPO"].shape

(590672,)

In [31]:
df[df["VIC_OCUP_GRUPO"]=="nan"]

,AGR_ALFAB,AGR_DEDICA,AGR_EDAD,AGR_ESCOLARIDAD,AGR_EST_CIV,AGR_GRUPET,AGR_NACIONAL,AGR_OCUP,AGR_SEXO,AGR_TRABAJA,...,VIC_OCUP_INE,AGR_OCUP_INE,VIC_OCUP_GRUPO,VIC_OCUP_SUB_GRUPO_PRINCIPAL,VIC_OCUP_SUB_GRUPO,VIC_OCUP_GRUPO_UNITARIO,AGR_OCUP_GRUPO,AGR_OCUP_SUB_GRUPO_PRINCIPAL,AGR_OCUP_SUB_GRUPO,AGR_OCUP_GRUPO_UNITARIO


### Unificar las variables VIC_DEDICA y VIC_OCUPACION (grupo)

In [ ]:
def obtener_vic_oficio(row):
    trabaja = row["VIC_TRABAJA"]
    if trabaja == "si" and row["VIC_OCUP_GRUPO"]!="Grupo ignorado":
        return row["VIC_OCUP_GRUPO"]
    dedica = row["VIC_DEDICA"]
    if dedica == "otro" and row["VIC_OCUP_GRUPO"]!="Grupo ignorado":
        return row["VIC_OCUP_GRUPO"]
    if dedica == "ignorado" and trabaja == "ignorado":
        return "ignorado"
    return dedica
df["VIC_OFICIO"] = df.apply(obtener_vic_oficio, axis=1)
def obtener_agr_oficio(row):
    trabaja = row["AGR_TRABAJA"]
    if trabaja == "si" and row["AGR_OCUP_GRUPO"]!="Grupo ignorado":
        return row["AGR_OCUP_GRUPO"]
    dedica = row["AGR_DEDICA"]
    if dedica == "otro" and row["AGR_OCUP_GRUPO"]!="Grupo ignorado":
        return row["AGR_OCUP_GRUPO"]
    if dedica == "ignorado" and trabaja == "ignorado":
        return "ignorado"
    return dedica
df["AGR_OFICIO"] = df.apply(obtener_agr_oficio, axis=1)

### Limpiar la relación entre víctimas y agresores

In [ ]:
df["VIC_REL_AGR"] = df["VIC_REL_AGR"].replace("esposos(a)", "esposos(as)")

/tmp/ipykernel_212406/815048184.py:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df["VIC_REL_AGR"] = df["VIC_REL_AGR"].replace("esposos(a)", "esposos(as)")


### Reemplazar valores nominales para los organismos jurisdireccionales que reciben la denuncia

In [36]:
organismo_jurisdiccional_map = {
    1.0: "Juzgados de Paz",
    2.0: "Juzgados de Paz Comunitarios",
    3.0: "Juzgados de Paz Penal",
    4.0: "Juzgado de Paz Penal de Conocimiento a Prevención de Delitos de Narcotráfico, Defraudación y Contrabando Aduanero, Violencia Sexual, Explotación y Trata de Personas",
    5.0: "Juzgados de Paz Penal de Turno/24horas",
    6.0: "Juzgados de Paz Móvil",
    7.0: "Juzgado de Paz con Competencia Específica para la Protección en Materia de Violencia Intrafamiliar, y de Niñez y Adolescencia Amenazada o Violada en sus Derechos (de turno)",
    8.0: "Juzgados de Primera Instancia Penal, Narcoactividad y Delitos contra el Ambiente",
    9.0: "Juzgados de Primera Instancia Penal, Narcoactividad y Delitos contra el Ambiente de Turno/24 horas",
    10.0: "Juzgado de Turno de Primera Instancia Penal de Delitos de Femicidio y otras formas de Violencia contra la contra la Mujer y Violencia Sexual, Explotación y Trata de Personas",
    11.0: "Juzgados de Primera Instancia Penal de Delitos de Femicidio y otras formas de Violencia contra la Mujer",
    12.0: "Tribunales de Sentencia Penal, Narcoactividad y Delitos contra el Ambiente",
    13.0: "Tribunales de Sentencia Penal de Delitos de Femicidio y otras formas de Violencia en contra de la Mujer",
    14.0: "Juzgados de Primera Instancia Ramo Mixto",
    15.0: "Juzgados de Primera Instancia de Familia",
    16.0: "Juzgados de Adolescentes en Conflicto con la Ley Penal",
    99.0: "Ignorado"
    
}
df["ORGANISMO_JURISDICCIONAL"] = df["ORGANISMO_JURISDICCIONAL"].map(organismo_jurisdiccional_map)

## Mapear los salarios según el tablero ENEI
https://www.ine.gob.gt/tablero-enei/
Lo que se realizó fue obtener el ingreso según la categoría por año, utilizando el trimestre último según el ENEI para los años que sí estaban existentes, y para los que no se utilizó el promedio de los últimos trimestres de los sí existentes.

In [37]:
df.HEC_ANO

0         2008.0
1         2007.0
2         2007.0
3         2008.0
4         2008.0
           ...  
590667    2023.0
590668    2023.0
590669    2023.0
590670    2023.0
590671    9999.0
Name: HEC_ANO, Length: 590672, dtype: float64

In [38]:
lista_tra = list(df['VIC_OFICIO'].unique())
lista_tra.extend(df['AGR_OFICIO'].unique())
set(lista_tra)

{'Directores y gerentes',
 'Ocupaciones elementales',
 'Ocupaciones militares',
 'Oficiales, Operarios y artesanos de artes mecánicas y de otros oficios',
 'Operadores de instalaciones y máquinas y ensambladores',
 'Personal de apoyo administrativo',
 'Profesionales científicos e intelectuales',
 'Trabajadores de los servicios y vendedores de comercios y mercados',
 'Técnicos y profesionales de nivel medio',
 'alcoholico/drogadicto',
 'desempleado',
 'estudiar',
 'ignorado',
 'otro',
 'quehaceres del hogar',
 'vive de sus renta o jubilacion'}

In [39]:

salarios = pd.DataFrame([
    ('Directores y gerentes', 5856.30,2014),
    ('Ocupaciones elementales', 1154.40, 2014),
    ('Ocupaciones militares', 1618.00,2014),
    ('Oficiales, Operarios y artesanos de artes mecánicas y de otros oficios', 2418.30, 2014),
    ('Operadores de instalaciones y máquinas y ensambladores', 2218.40,2014),
    ('Personal de apoyo administrativo', 3228.70,2014),
    ('Profesionales científicos e intelectuales', 3228.70,2014),
    ('Trabajadores de los servicios y vendedores de comercios y mercados', 2472.00,2014),
    ('Técnicos y profesionales de nivel medio', 3228.70,2014),
    ('vive de sus renta o jubilacion', 6294.40, 2014),
    ('quehaceres del hogar', 754.10, 2014),
    ('otro', 1618.00, 2014),
    
    
    ('Directores y gerentes', 6483.40, 2015),
    ('Ocupaciones elementales', 1166.20, 2015),
    ('Ocupaciones militares', 1554.50, 2015),
    ('Oficiales, Operarios y artesanos de artes mecánicas y de otros oficios', 2601.20, 2015),
    ('Operadores de instalaciones y máquinas y ensambladores', 2441.80, 2015),
    ('Personal de apoyo administrativo', 3363.40, 2015),
    ('Profesionales científicos e intelectuales', 3363.40, 2015),
    ('Trabajadores de los servicios y vendedores de comercios y mercados', 2462.60, 2015),
    ('Técnicos y profesionales de nivel medio', 3363.40, 2015),
    ('vive de sus renta o jubilacion', 3486.30, 2015),
    ('quehaceres del hogar', 969.80, 2015),
    ('otro', 1554.50, 2015),
    
    ('Directores y gerentes', 7297.40, 2016),
    ('Ocupaciones elementales', 1050.30, 2016),
    ('Ocupaciones militares', 1302.00, 2016),
    ('Oficiales, Operarios y artesanos de artes mecánicas y de otros oficios', 2298.40, 2016),
    ('Operadores de instalaciones y máquinas y ensambladores', 2366.90, 2016),
    ('Personal de apoyo administrativo', 3113.80, 2016),
    ('Profesionales científicos e intelectuales', 3113.80, 2016),
    ('Trabajadores de los servicios y vendedores de comercios y mercados', 2498.80, 2016),
    ('Técnicos y profesionales de nivel medio', 3113.80, 2016),
    ('vive de sus renta o jubilacion', 3418.10, 2016),
    ('quehaceres del hogar', 894.40, 2016),
    ('otro', 1501.50, 2016),
    
    ('Directores y gerentes', 8285.10, 2017),
    ('Ocupaciones elementales', 1173.90, 2017),
    ('Ocupaciones militares', 1334.90, 2017),
    ('Oficiales, Operarios y artesanos de artes mecánicas y de otros oficios', 2127.00, 2017),
    ('Operadores de instalaciones y máquinas y ensambladores', 2173.80, 2017),
    ('Personal de apoyo administrativo', 3257.80, 2017),
    ('Profesionales científicos e intelectuales', 3257.80, 2017),
    ('Trabajadores de los servicios y vendedores de comercios y mercados', 2567.90, 2017),
    ('Técnicos y profesionales de nivel medio', 3257.80, 2017),
    ('vive de sus renta o jubilacion', 5066.60, 2017),
    ('quehaceres del hogar', 836.50, 2017),
    ('otro', 1334.90, 2017),
    
    ('Directores y gerentes', 8378.40, 2018),
    ('Ocupaciones elementales', 1178.90, 2018),
    ('Ocupaciones militares', 1456.20, 2018),
    ('Oficiales, Operarios y artesanos de artes mecánicas y de otros oficios', 2152.80, 2018),
    ('Operadores de instalaciones y máquinas y ensambladores', 2436.40, 2018),
    ('Personal de apoyo administrativo', 3450.90, 2018),
    ('Profesionales científicos e intelectuales', 3450.90, 2018),
    ('Trabajadores de los servicios y vendedores de comercios y mercados', 2681.80, 2018),
    ('Técnicos y profesionales de nivel medio', 3450.90, 2018),
    ('vive de sus renta o jubilacion', 4426.20, 2018),
    ('quehaceres del hogar', 898.30, 2018),
    ('otro', 1456.20, 2018),
    
    ('Directores y gerentes', 7295.10, 2019),
    ('Ocupaciones elementales', 1505.10, 2019),
    ('Ocupaciones militares', 1584.40, 2019),
    ('Oficiales, Operarios y artesanos de artes mecánicas y de otros oficios', 2297.00, 2019),
    ('Operadores de instalaciones y máquinas y ensambladores', 2478.40, 2019),
    ('Personal de apoyo administrativo', 3890.20, 2019),
    ('Profesionales científicos e intelectuales', 3890.20, 2019),
    ('Trabajadores de los servicios y vendedores de comercios y mercados', 2659.00, 2019),
    ('Técnicos y profesionales de nivel medio', 3890.20, 2019),
    
    ('vive de sus renta o jubilacion', 4396.40, 2019),
    ('quehaceres del hogar', 1033.40, 2019),
    ('otro', 1584.40, 2019),
    
    ('Directores y gerentes', 4325.90, 2021),
    ('Ocupaciones elementales', 1563.20, 2021),
    ('Ocupaciones militares', 1430.90, 2021),
    ('Oficiales, Operarios y artesanos de artes mecánicas y de otros oficios', 2221.40, 2021),
    ('Operadores de instalaciones y máquinas y ensambladores', 1887.80, 2021),
    ('Personal de apoyo administrativo', 3908.00, 2021),
    ('Profesionales científicos e intelectuales', 3908.00, 2021),
    ('Trabajadores de los servicios y vendedores de comercios y mercados', 2229.50, 2021),
    ('Técnicos y profesionales de nivel medio', 3908.00, 2021),   
    
    ('desempleado', 0.00, 2021),
    ('estudiar', -100.00, 2021), # Asumiendo costos USAC
    ('alcoholico/drogadicto', 1159, 2021), # Basado en un estudio
    ('vive de sus renta o jubilacion', 3251.20, 2021),
    ('quehaceres del hogar', 993.00, 2021),
    ('otro', 1430.90, 2021),
    
],
columns=['Ocupacion', 'Salario', 'Anio']
)
round(salarios[['Ocupacion', 'Salario']].groupby('Ocupacion').mean(), 2).reset_index()

,Ocupacion,Salario
0,Directores y gerentes,6845.94
1,Ocupaciones elementales,1256.00
2,Ocupaciones militares,1468.70
3,"Oficiales, Operarios y artesanos de artes mecá...",2302.30
4,Operadores de instalaciones y máquinas y ensam...,2286.21
5,Personal de apoyo administrativo,3458.97
6,Profesionales científicos e intelectuales,3458.97
7,Trabajadores de los servicios y vendedores de ...,2510.23
8,Técnicos y profesionales de nivel medio,3458.97
9,alcoholico/drogadicto,1159.00


In [ ]:
df = df.merge(
    salarios.rename(columns={'Ocupacion': 'VIC_OFICIO', 'Anio': 'HEC_ANO', 'Salario': 'VIC_SALARIO'}),
    on=['VIC_OFICIO', 'HEC_ANO'],
    how='left'
)
salario_mean = salarios.groupby('Ocupacion')['Salario'].mean().round(2).reset_index()
df = df.merge(
    salario_mean.rename(columns={'Ocupacion': 'VIC_OFICIO', 'Salario': 'SALARIO_MEAN'}),
    on='VIC_OFICIO',
    how='left'
)

df['VIC_SALARIO'] = df['VIC_SALARIO'].fillna(df['SALARIO_MEAN'])
df.drop(columns=['SALARIO_MEAN'], inplace=True)


In [ ]:
df = df.merge(
    salarios.rename(columns={'Ocupacion': 'AGR_OFICIO', 'Anio': 'HEC_ANO', 'Salario': 'AGR_SALARIO'}),
    on=['AGR_OFICIO', 'HEC_ANO'],
    how='left'
)
salario_mean = salarios.groupby('Ocupacion')['Salario'].mean().round(2).reset_index()
df = df.merge(
    salario_mean.rename(columns={'Ocupacion': 'AGR_OFICIO', 'Salario': 'SALARIO_MEAN'}),
    on='AGR_OFICIO',
    how='left'
)

df['AGR_SALARIO'] = df['AGR_SALARIO'].fillna(df['SALARIO_MEAN'])
df.drop(columns=['SALARIO_MEAN'], inplace=True)


## Encoding

### One Hot Encoding
Esto para columnas que contengan categorías menor o igual que 10.

In [42]:
# def get_columns_with_10_categories(non_numerical_cols):
#     columns = []
#     for col in non_numerical_cols:
#         if len(df[col].unique()) <= 10:
#             columns.append(col)
#     return columns
# one_hot_encoding_cols = get_columns_with_10_categories(non_numerical)

In [43]:
# batch_size = 10000  # Adjust based on your memory constraints
# dummies_list = []
# dummies_col_list = []
# for i in range(0, len(one_hot_encoding_cols), batch_size):
#     batch = one_hot_encoding_cols[i:i + batch_size]
#     dummies = pd.get_dummies(df[batch], drop_first=True)
#     dummies_list.append(dummies)
#     dummies_col_list.extend(dummies.columns)
# df_dummies = pd.concat(dummies_list, axis=1)
# df = pd.concat([df, df_dummies], axis=1)
# df[dummies_col_list] = df[dummies_col_list].replace(True, 1)
# df[dummies_col_list] = df[dummies_col_list].replace(False, 0)

### Label Encoding
Aquellas categorías que se extienden a más de 10 categorías.

In [44]:
# encodings = {}
# encoding_cols = [col for col in non_numerical if col not in one_hot_encoding_cols]
# for col in encoding_cols:
#     le = preprocessing.LabelEncoder()
#     df["label_encoding_"+col] = le.fit_transform(df[col])
#     encodings[col] = le

## Guardar el dataframe con las variables

In [ ]:
df.to_csv(f'./dataset/violenciaIntrafamiliarGuatemala2008_2023_v2.csv', index=False)